In [273]:
#Open Excel File
import openpyxl
import os
import pandas as pd
import numpy as np
import re 

# Load the workbook
workbook = openpyxl.load_workbook('../1st_dataset.xlsx')
worksheet = workbook["Sheet1"]

data = []
headers = []

# Get headers from the first row
for col in range(1, worksheet.max_column + 1):
    headers.append(worksheet.cell(row=1, column=col).value)

# Get data from remaining rows
for row in range(2, worksheet.max_row + 1):
    row_data = []
    for col in range(1, worksheet.max_column + 1):
        row_data.append(worksheet.cell(row=row, column=col).value)
    data.append(row_data)

# Create DataFrame
df = pd.DataFrame(data, columns=headers)

print(headers)

# Display first few rows to verify
print(f"Dataset shape: {df.shape}")
df.head()

['name', 'ingredients', 'ner_ingredient', 'instructions', 'min_age', 'max_age', 'texture', 'prep_time', 'cook_time', 'serving', 'origin', 'recipe_link', 'credibility', 'image_link', 'region', 'difficulty', 'meal_type', 'description', 'dietary_tags', 'choking_hazard', 'tips', 'allergen', 'hypoallergenic', 'nutrition_value', 'ID', 'Energy / Calorie', 'Carbohydrate (g)', 'Protein (g)', 'Fat (g)', 'List of Micros', None, None, None, None, None]
Dataset shape: (1322, 35)


,name,ingredients,ner_ingredient,instructions,min_age,max_age,texture,prep_time,cook_time,serving,...,Energy / Calorie,Carbohydrate (g),Protein (g),Fat (g),List of Micros,None,None,None,None,None
0,Cassava Porridge with Fish Sauce and Lemon (Bu...,"- 60 g cassava, boiled and blended\n- 20 g fis...","['cassava', 'fish', 'chicken', 'coconut oil', ...","Broth:\n1. Use chicken bones, chicken feet, fi...",6,8,None,15 min,45 min,1,...,None,None,None,None,None,None,None,None,None,None
1,Bitterballs (Bitterballen),- 100 g beef mince \n- 30 g potato starch \n- ...,"['beef', 'potato starch', 'milk', 'egg', 'marg...",1. Stir-fry blended spices until fragrant. \n2...,9,11,None,30 minutes,30 minutes,10 servings,...,None,None,None,None,None,None,None,None,None,None
2,Broccoli/Cauliflower Cheese,"- 175g cauliflower/broccoli, cut into pieces\n...","['cauliflower', 'broccoli', 'margarine', 'flou...","1. Steam, boil, or microwave cauliflower/brocc...",6,12,None,~10 min,~20 min,None,...,None,None,None,None,None,None,None,None,None,None
3,Vegetable Fingers,"- 1 carrot, potato, or sweet potato, peeled an...","['carrot', 'potato', 'sweet potato']",1. Steam or microwave vegetables until tender....,6,12,None,~5 min,~10 min,None,...,None,None,None,None,None,None,None,None,None,None
4,Beef Casserole,"- 1 onion, peeled and finely chopped\n- 1½ tab...","['onion', 'vegetable oil', 'beef', 'steak', 'c...",1. Preheat oven to 180°C.\n2. Heat oil in a me...,6,12,None,~10 min,~2.5 hours,None,...,None,None,None,None,None,None,None,None,None,None


In [274]:
# Clean up None columns first
none_columns = [col for col in df.columns if col is None]
if none_columns:
    df = df.drop(columns=none_columns)
    print(f"Dropped {len(none_columns)} None column(s)")


Dropped 5 None column(s)


In [275]:
# Check which records have empty nutrition_value
empty_nutrition = df['nutrition_value'].isna() | (df['nutrition_value'] == '') | (df['nutrition_value'] == 'None')
print(f"Records with empty nutrition_value: {empty_nutrition.sum()} out of {len(df)}")


# Function to extract ingredient information
def extract_ingredient_info(ingredient_text):
    """Extract quantity, measurement, and ingredient name from ingredient text"""
    if pd.isna(ingredient_text) or ingredient_text == '':
        return []
    
    # Split by lines and clean
    lines = [line.strip() for line in ingredient_text.split('\n') if line.strip()]
    
    extracted_ingredients = []
    
    for line in lines:
        # Remove leading dash or bullet points
        line = re.sub(r'^[-•*]\s*', '', line)
        
        # Pattern to match quantity, measurement, and ingredient
        # Examples: "60 g cassava", "2-3 tablespoons of plain yogurt", "1½ tablespoons vegetable oil"
        patterns = [
            # Pattern 1: Range + unit + "of" + ingredient (e.g., "2-3 tablespoons of plain yogurt")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?[-–][0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s*([a-zA-Z]+)\s+of\s+(.+)$',
            
            # Pattern 2: Range + unit + ingredient (e.g., "2-3 tablespoons plain yogurt")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?[-–][0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s*([a-zA-Z]+)\s+(.+)$',
            
            # Pattern 3: Number + unit + "of" + ingredient (e.g., "30 g of sweet potato")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s*([a-zA-Z]+)\s+of\s+(.+)$',
            
            # Pattern 4: Number + unit + ingredient (e.g., "60 g cassava", "175g cauliflower")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s*([a-zA-Z]+)\s+(.+)$',
            
            # Pattern 5: Fraction + unit + "of" + ingredient (e.g., "1½ tablespoons of oil")
            r'^([0-9]*[½¼¾][0-9]*)\s*([a-zA-Z]+)\s+of\s+(.+)$',
            
            # Pattern 6: Fraction + unit + ingredient (e.g., "1½ tablespoons oil")
            r'^([0-9]*[½¼¾][0-9]*)\s*([a-zA-Z]+)\s+(.+)$',
            
            # Pattern 7: Range + ingredient (no unit) (e.g., "2-3 carrots")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?[-–][0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s+(.+)$',
            
            # Pattern 8: Number + ingredient (no unit) (e.g., "1 carrot", "1 onion")
            r'^([0-9]+(?:[.,][0-9]+)?(?:[½¼¾])?)\s+(.+)$',
            
            # Pattern 9: Just ingredient (no quantity/unit)
            r'^(.+)$'
        ]
        
        quantity = None
        measurement = None
        ingredient_name = None
        
        for i, pattern in enumerate(patterns):
            match = re.match(pattern, line, re.IGNORECASE)
            if match:
                if i == 0:  # Pattern 1: range + unit + "of" + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 1:  # Pattern 2: range + unit + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 2:  # Pattern 3: number + unit + "of" + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 3:  # Pattern 4: number + unit + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 4:  # Pattern 5: fraction + unit + "of" + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 5:  # Pattern 6: fraction + unit + ingredient
                    quantity = match.group(1)
                    measurement = match.group(2)
                    ingredient_name = match.group(3).strip()
                elif i == 6:  # Pattern 7: range + ingredient (no unit)
                    quantity = match.group(1)
                    measurement = None
                    ingredient_name = match.group(2).strip()
                elif i == 7:  # Pattern 8: number + ingredient (no unit)
                    quantity = match.group(1)
                    measurement = None
                    ingredient_name = match.group(2).strip()
                else:  # Pattern 9: just ingredient
                    quantity = None
                    measurement = None
                    ingredient_name = match.group(1).strip()
                break
        
        # Clean up ingredient name (remove extra descriptions after comma)
        if ingredient_name:
            # Remove descriptions after comma (e.g., "cassava, boiled and blended" -> "cassava")
            ingredient_name = ingredient_name.split(',')[0].strip()
            
            extracted_ingredients.append({
                'original_text': line,
                'quantity': quantity,
                'measurement': measurement,
                'ingredient_name': ingredient_name
            })
    
    return extracted_ingredients



Records with empty nutrition_value: 446 out of 1322


In [276]:
# Test the function with specific examples
print("=== TESTING INGREDIENT EXTRACTION WITH RANGE FORMATS ===")

test_ingredients = [
    "2-3 tablespoons of plain yogurt",
    "1-2 teaspoons of vanilla extract", 
    "30 g of sweet potato",
    "60 g cassava",
    "1½ tablespoons vegetable oil",
    "2-3 carrots",
    "1 onion",
    "plain water"
]

for test_ingredient in test_ingredients:
    print(f"\nTesting: '{test_ingredient}'")
    result = extract_ingredient_info(test_ingredient)
    if result:
        for r in result:
            print(f"  ✅ Quantity: {r['quantity']}, Measurement: {r['measurement']}, Ingredient: {r['ingredient_name']}")
    else:
        print(f"  ❌ No match found")


=== TESTING INGREDIENT EXTRACTION WITH RANGE FORMATS ===

Testing: '2-3 tablespoons of plain yogurt'
  ✅ Quantity: 2-3, Measurement: tablespoons, Ingredient: plain yogurt

Testing: '1-2 teaspoons of vanilla extract'
  ✅ Quantity: 1-2, Measurement: teaspoons, Ingredient: vanilla extract

Testing: '30 g of sweet potato'
  ✅ Quantity: 30, Measurement: g, Ingredient: sweet potato

Testing: '60 g cassava'
  ✅ Quantity: 60, Measurement: g, Ingredient: cassava

Testing: '1½ tablespoons vegetable oil'
  ✅ Quantity: 1½, Measurement: tablespoons, Ingredient: vegetable oil

Testing: '2-3 carrots'
  ✅ Quantity: 2-3, Measurement: None, Ingredient: carrots

Testing: '1 onion'
  ✅ Quantity: 1, Measurement: None, Ingredient: onion

Testing: 'plain water'
  ✅ Quantity: None, Measurement: None, Ingredient: plain water


In [277]:
df_empty_nutrition = df[df['nutrition_value'].isna() | (df['nutrition_value'] == '') | (df['nutrition_value'] == 'None')].copy()

print(f"\nExtracting ingredient information for {len(df_empty_nutrition)} records with empty nutrition_value...")
df_empty_nutrition['extracted_ingredients'] = df_empty_nutrition['ingredients'].apply(extract_ingredient_info)

df_empty_nutrition[['ingredients','extracted_ingredients']]



Extracting ingredient information for 446 records with empty nutrition_value...


,ingredients,extracted_ingredients
0,"- 60 g cassava, boiled and blended\n- 20 g fis...","[{'original_text': '60 g cassava, boiled and b..."
1,- 100 g beef mince \n- 30 g potato starch \n- ...,"[{'original_text': '100 g beef mince', 'quanti..."
2,"- 175g cauliflower/broccoli, cut into pieces\n...","[{'original_text': '175g cauliflower/broccoli,..."
3,"- 1 carrot, potato, or sweet potato, peeled an...","[{'original_text': '1 carrot, potato, or sweet..."
4,"- 1 onion, peeled and finely chopped\n- 1½ tab...","[{'original_text': '1 onion, peeled and finely..."
...,...,...
1317,- 50 g berries (you can use a mix of frozen be...,[{'original_text': '50 g berries (you can use ...
1318,"- 3 small mushrooms, finely chopped\n- ½ cup b...","[{'original_text': '3 small mushrooms, finely ..."
1319,- 1 cup pasta\n- 1 tablespoon margarine\n- 1 t...,"[{'original_text': '1 cup pasta', 'quantity': ..."
1320,- ¼ cup sugar\n- 1 cup milk\n- 1 egg\n- 2 tabl...,"[{'original_text': '¼ cup sugar', 'quantity': ..."


In [278]:
# Display sample extractions
print("\nSample ingredient extractions:")
for idx, row in df_empty_nutrition.head(3).iterrows():
    print(f"\n--- Recipe: {row['name']} ---")
    print(f"Original ingredients:\n{row['ingredients']}")
    print("\nExtracted ingredients:")
    for ing in row['extracted_ingredients']:
        print(f"  - Quantity: {ing['quantity']}, Measurement: {ing['measurement']}, Ingredient: {ing['ingredient_name']}")
        print(f"    Original: {ing['original_text']}")



Sample ingredient extractions:

--- Recipe: Cassava Porridge with Fish Sauce and Lemon (Bubur Singkong Kukuruyuk Saus Jeruk) ---
Original ingredients:
- 60 g cassava, boiled and blended
- 20 g fish meat (milkfish), finely chopped
- 10 g chicken meat
- 5 g coconut oil
- 100 cc chicken broth
- 10 g fresh lime juice
- 20 g spinach, finely chopped

Extracted ingredients:
  - Quantity: 60, Measurement: g, Ingredient: cassava
    Original: 60 g cassava, boiled and blended
  - Quantity: 20, Measurement: g, Ingredient: fish meat (milkfish)
    Original: 20 g fish meat (milkfish), finely chopped
  - Quantity: 10, Measurement: g, Ingredient: chicken meat
    Original: 10 g chicken meat
  - Quantity: 5, Measurement: g, Ingredient: coconut oil
    Original: 5 g coconut oil
  - Quantity: 100, Measurement: cc, Ingredient: chicken broth
    Original: 100 cc chicken broth
  - Quantity: 10, Measurement: g, Ingredient: fresh lime juice
    Original: 10 g fresh lime juice
  - Quantity: 20, Measurement: 

In [279]:
# Create a detailed breakdown DataFrame
ingredient_details = []
for idx, row in df_empty_nutrition.iterrows():
    recipe_name = row['name']
    for ing in row['extracted_ingredients']:
        ingredient_details.append({
            'recipe_id': idx,
            'recipe_name': recipe_name,
            'quantity': ing['quantity'],
            'measurement': ing['measurement'],
            'ingredient_name': ing['ingredient_name'],
            'original_text': ing['original_text']
        })

ingredient_breakdown_df = pd.DataFrame(ingredient_details)
print(f"\nCreated ingredient breakdown with {len(ingredient_breakdown_df)} ingredient entries")
print(f"From {len(df_empty_nutrition)} recipes with empty nutrition values")




Created ingredient breakdown with 2572 ingredient entries
From 446 recipes with empty nutrition values


In [280]:
#print new df
ingredient_breakdown_df.head()


,recipe_id,recipe_name,quantity,measurement,ingredient_name,original_text
0,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,60,g,cassava,"60 g cassava, boiled and blended"
1,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,20,g,fish meat (milkfish),"20 g fish meat (milkfish), finely chopped"
2,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,10,g,chicken meat,10 g chicken meat
3,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,5,g,coconut oil,5 g coconut oil
4,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,100,cc,chicken broth,100 cc chicken broth


In [281]:
# Show summary statistics
print("\nSummary of extracted measurements:")
if len(ingredient_breakdown_df) > 0:
    print(ingredient_breakdown_df['measurement'].value_counts().head(10))
    
    print("\nMost common ingredients:")
    print(ingredient_breakdown_df['ingredient_name'].value_counts().head(10))


Summary of extracted measurements:
measurement
g              556
tsp            181
small          148
tbsp           130
cup             85
ml              79
medium          73
tablespoons     47
teaspoon        45
tablespoon      43
Name: count, dtype: int64

Most common ingredients:
ingredient_name
water            69
vegetable oil    65
onion            48
egg              38
carrot           35
olive oil        34
butter           29
potato           25
eggs             25
sugar            20
Name: count, dtype: int64


In [282]:
# Function to convert measurements to grams
def convert_to_grams(quantity, measurement, ingredient_name):
    """
    Convert quantity and measurement to grams based on common cooking conversions.
    Returns converted quantity in grams and 'g' as measurement.
    """
    if not quantity or not measurement:
        return quantity, measurement
    
    # Clean and parse quantity (handle ranges and fractions)
    def parse_quantity(qty_str):
        if not qty_str:
            return 1.0
        
        # Handle fractions
        fraction_map = {'½': 0.5, '¼': 0.25, '¾': 0.75, '⅓': 0.33, '⅔': 0.67}
        for frac, val in fraction_map.items():
            qty_str = str(qty_str).replace(frac, str(val))
        
        # Handle ranges (take average)
        if '-' in str(qty_str) or '–' in str(qty_str):
            parts = re.split(r'[-–]', str(qty_str))
            if len(parts) == 2:
                try:
                    min_val = float(parts[0].strip())
                    max_val = float(parts[1].strip())
                    return (min_val + max_val) / 2
                except:
                    pass
        
        # Convert to float
        try:
            return float(qty_str)
        except:
            return 1.0
    
    parsed_qty = parse_quantity(quantity)
    measurement_lower = str(measurement).lower()
    
    # Conversion factors to grams
    conversions = {
        # Volume conversions (approximate for common ingredients)
        'tsp': 5,           # 1 tsp ≈ 5g (for most liquids/powders)
        'teaspoon': 5,
        'teaspoons': 5,
        'tbsp': 15,         # 1 tbsp ≈ 15g
        'tablespoon': 15,
        'tablespoons': 15,
        'cup': 240,         # 1 cup ≈ 240g (for liquids)
        'cups': 240,
        'ml': 1,            # 1ml ≈ 1g (for water-based liquids)
        'milliliters': 1,
        'milliliter': 1,
        'l': 1000,          # 1 liter = 1000g
        'liter': 1000,
        'liters': 1000,
        
        # Weight conversions
        'g': 1,             # already in grams
        'gram': 1,
        'grams': 1,
        'kg': 1000,         # 1 kg = 1000g
        'kilogram': 1000,
        'kilograms': 1000,
        'oz': 28.35,        # 1 oz ≈ 28.35g
        'ounce': 28.35,
        'ounces': 28.35,
        'lb': 453.6,        # 1 lb ≈ 453.6g
        'pound': 453.6,
        'pounds': 453.6,
        'cc': 1,           
        
        # Piece conversions (rough estimates)
        'small': 50,        # small piece ≈ 50g
        'medium': 100,      # medium piece ≈ 100g
        'large': 150,       # large piece ≈ 150g
        'piece': 75,        # average piece ≈ 75g
        'pieces': 75,
        'clove': 3,         # garlic clove ≈ 3g
        'cloves': 3,
    }
    
    # Convert to grams
    if measurement_lower in conversions:
        converted_qty = parsed_qty * conversions[measurement_lower]
        return round(converted_qty, 1), 'g'
    else:
        # If measurement not found, return original
        return quantity, measurement



In [283]:
# Test the conversion function
print("Testing conversion function:")
test_cases = [
    ('2', 'tbsp', 'vegetable oil'),
    ('1', 'cup', 'flour'),
    ('1', 'cc', 'salt'),
    ('100', 'ml', 'water'),
    ('1', 'medium', 'onion'),
    ('2-3', 'tsp', 'sugar'),
    ('½', 'cup', 'butter')
]

# for qty, measure, ingredient in test_cases:
#     # new_qty, new_measure = convert_to_grams(qty, measure, ingredient)
#     print(f"{qty} {measure} {ingredient} → {new_qty} {new_measure}")


Testing conversion function:


In [284]:
# Apply conversion to all ingredients in the dataframe
print("Converting all measurements to grams...")

# Create new columns for converted values
ingredient_breakdown_df['original_quantity'] = ingredient_breakdown_df['quantity'].copy()
ingredient_breakdown_df['original_measurement'] = ingredient_breakdown_df['measurement'].copy()

# # Apply conversion
# conversion_results = ingredient_breakdown_df.apply(
#     lambda row: convert_to_grams(row['quantity'], row['measurement'], row['ingredient_name']), 
#     axis=1
# )

# Update the dataframe with converted values
ingredient_breakdown_df['quantity'] = [result[0] for result in conversion_results]
ingredient_breakdown_df['measurement'] = [result[1] for result in conversion_results]

print(f"\nConversion completed for {len(ingredient_breakdown_df)} ingredients")

# Show sample conversions
print("\nSample conversions:")
sample_conversions = ingredient_breakdown_df[ingredient_breakdown_df['original_measurement'].notna()].head(10)
for idx, row in sample_conversions.iterrows():
    print(f"{row['original_quantity']} {row['original_measurement']} {row['ingredient_name']} → {row['quantity']} {row['measurement']}")

# Show new measurement distribution
print("\nNew measurement distribution:")
print(ingredient_breakdown_df['measurement'].value_counts())



Converting all measurements to grams...

Conversion completed for 2572 ingredients

Sample conversions:
60 g cassava → 60.0 g
20 g fish meat (milkfish) → 20.0 g
10 g chicken meat → 10.0 g
5 g coconut oil → 5.0 g
100 cc chicken broth → 100.0 g
10 g fresh lime juice → 10.0 g
20 g spinach → 20.0 g
100 g beef mince → 100.0 g
30 g potato starch → 30.0 g
300 ml milk → 300.0 g

New measurement distribution:
measurement
g           1548
heaped        16
knob          14
pinch         12
level         11
            ... 
lamb           1
pot            1
macarons       1
knobs          1
leaves         1
Name: count, Length: 126, dtype: int64


In [285]:
#print new df
ingredient_breakdown_df.head()

,recipe_id,recipe_name,quantity,measurement,ingredient_name,original_text,original_quantity,original_measurement
0,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,60.0,g,cassava,"60 g cassava, boiled and blended",60,g
1,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,20.0,g,fish meat (milkfish),"20 g fish meat (milkfish), finely chopped",20,g
2,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,10.0,g,chicken meat,10 g chicken meat,10,g
3,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,5.0,g,coconut oil,5 g coconut oil,5,g
4,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,100.0,g,chicken broth,100 cc chicken broth,100,cc


In [286]:
# Filter ingredients that have complete quantity and measurement data
print("Filtering ingredients with complete quantity and measurement data...")

# Check current data completeness
print(f"\nTotal ingredients in breakdown: {len(ingredient_breakdown_df)}")
print(f"Ingredients with quantity: {ingredient_breakdown_df['quantity'].notna().sum()}")
print(f"Ingredients with measurement: {ingredient_breakdown_df['measurement'].notna().sum()}")
print(f"Ingredients with both quantity and measurement: {(ingredient_breakdown_df['quantity'].notna() & ingredient_breakdown_df['measurement'].notna()).sum()}")

# Create mask for complete data (both quantity and measurement are not null)
complete_data_mask = (
    ingredient_breakdown_df['quantity'].notna() & 
    ingredient_breakdown_df['measurement'].notna() &
    (ingredient_breakdown_df['quantity'] != '') &
    (ingredient_breakdown_df['measurement'] != '')
)

# Filter dataframe to only include ingredients with complete data
ingredients_complete = ingredient_breakdown_df[complete_data_mask].copy()

print(f"\nIngredients with complete quantity and measurement: {len(ingredients_complete)}")
print(f"Percentage of complete data: {len(ingredients_complete) / len(ingredient_breakdown_df) * 100:.1f}%")

# Show sample of complete ingredients
print("\nSample of ingredients with complete data:")
print(ingredients_complete[['ingredient_name', 'quantity', 'measurement']].head(10))


Filtering ingredients with complete quantity and measurement data...

Total ingredients in breakdown: 2572
Ingredients with quantity: 2061
Ingredients with measurement: 1895
Ingredients with both quantity and measurement: 1895

Ingredients with complete quantity and measurement: 1895
Percentage of complete data: 73.7%

Sample of ingredients with complete data:
        ingredient_name quantity measurement
0               cassava     60.0           g
1  fish meat (milkfish)     20.0           g
2          chicken meat     10.0           g
3           coconut oil      5.0           g
4         chicken broth    100.0           g
5      fresh lime juice     10.0           g
6               spinach     20.0           g
7            beef mince    100.0           g
8         potato starch     30.0           g
9                  milk    300.0           g


In [287]:
ingredient_breakdown_df[complete_data_mask]

,recipe_id,recipe_name,quantity,measurement,ingredient_name,original_text,original_quantity,original_measurement
0,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,60.0,g,cassava,"60 g cassava, boiled and blended",60,g
1,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,20.0,g,fish meat (milkfish),"20 g fish meat (milkfish), finely chopped",20,g
2,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,10.0,g,chicken meat,10 g chicken meat,10,g
3,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,5.0,g,coconut oil,5 g coconut oil,5,g
4,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,100.0,g,chicken broth,100 cc chicken broth,100,cc
...,...,...,...,...,...,...,...,...
2567,1321,Apple Crumble,60.0,g,wholemeal self-raising flour,¼ cup wholemeal self-raising flour,¼,cup
2568,1321,Apple Crumble,60.0,g,brown sugar,¼ cup brown sugar,¼,cup
2569,1321,Apple Crumble,60.0,g,coconut,¼ cup coconut,¼,cup
2570,1321,Apple Crumble,60.0,g,rolled oats,¼ cup rolled oats,¼,cup


In [288]:
ingredient_counts = ingredients_complete['ingredient_name'].value_counts()
print(f"Total unique ingredients: {len(ingredient_counts)}")
print("\nMost frequently used ingredients:")
print(ingredient_counts.head(15))

# Create a dataframe with ingredient frequencies
unique_ingredients_with_counts = ingredient_counts.reset_index()
unique_ingredients_with_counts.columns = ['ingredient_name', 'frequency']
print(f"\nCreated dataframe with {len(unique_ingredients_with_counts)} unique ingredients")



Total unique ingredients: 873

Most frequently used ingredients:
ingredient_name
water                   68
vegetable oil           65
onion                   41
olive oil               34
butter                  29
carrot                  23
sugar                   20
garlic                  19
frozen peas             16
potatoes                14
vegetable oil spread    13
flour                   13
lemon juice             12
milk                    12
potato                  12
Name: count, dtype: int64

Created dataframe with 873 unique ingredients


In [289]:

unique_ingredients_with_counts


,ingredient_name,frequency
0,water,68
1,vegetable oil,65
2,onion,41
3,olive oil,34
4,butter,29
...,...,...
868,Natto,1
869,Japanese rice *steamed,1
870,dashi (baby-safe,1
871,udon (preferably thin,1


## Requesting to USDA Food Central

In [290]:
import requests
from tqdm import tqdm
import requests
import time

api_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
api_key = "KulngHmZ1nJeaPPBrZ8pH3kyJI2Gy1r9Xm121YO9"

def get_nutrition_data(query: str, api_key: str):
    """
    Fetch nutrition data for a specific ingredient from USDA API with prioritized search strategy.
    
    Search Strategy:
    1. Foundation exact match
    2. Foundation first result
    3. Survey (FNDDS) exact match  
    4. Survey (FNDDS) first result
    
    Returns:
    dict: Nutrition information including energy, macronutrients, top 3 micronutrients by value,
          and search method used
    """
    
    base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    
    def search_with_params(data_type):
        """Helper function to search with specific parameters"""
        params = {
            'query': query,
            'api_key': api_key,
            'dataType': [data_type],
            'pageSize': 25,  # Get more results for better matching
            'pageNumber': 1,
            'sortBy': 'dataType.keyword',
            'sortOrder': 'asc'
        }
        
        try:
            response = requests.get(base_url, params=params, timeout=10)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"❌ API request failed: {response.status_code}")
                return None
        except Exception as e:
            print(f"❌ Error in API request: {e}")
            return None
    
    def find_exact_match(foods, query_lower):
        """Find exact match by comparing full ingredient name with full food description (case-insensitive)"""
        for food in foods:
            description_lower = food['description'].lower()
            # Check if the query exactly matches the description or if query is a whole word in description
            # Use word boundaries to ensure exact matching
            import re
            
            # Create pattern for exact word matching
            pattern = r'\b' + re.escape(query_lower) + r'\b'
            
            # Check if query is the entire description OR appears as complete word(s)
            if (query_lower == description_lower or 
                re.search(pattern, description_lower)):
                return food
        return None
    
    def extract_nutrition_info(food, search_method):
        """Extract nutrition information from food item"""
        nutrients = food.get('foodNutrients', [])
        
        result = {
            'ingredient_name': query,
            'found_description': food['description'],
            'search_method': search_method,
            'energy_kcal': None,
            'carbohydrate_g': None,
            'protein_g': None,
            'fat_g': None,
            'micronutrients': [],
            'status': 'success'
        }
        
        # Energy (Atwater General Factors)
        energy = next((item for item in nutrients if item['nutrientName'] == 'Energy (Atwater Specific Factors)'), None)
        if energy:
            result['energy_kcal'] = energy['value']
        
        # Carbohydrates
        carbohydrate = next((item for item in nutrients if item['nutrientName'] == 'Carbohydrate, by difference'), None)
        if carbohydrate:
            result['carbohydrate_g'] = carbohydrate['value']
        
        # Fat
        fat = next((item for item in nutrients if item['nutrientName'] == 'Total lipid (fat)'), None)
        if fat:
            result['fat_g'] = fat['value']
        
        # Protein
        protein = next((item for item in nutrients if item['nutrientName'] == 'Protein'), None)
        if protein:
            result['protein_g'] = protein['value']
        
        # Exclude certain nutrients from micronutrients
        exclude_nutrients = [
            "Energy", "Water", "Energy (Atwater General Factors)", "Energy (Atwater Specific Factors)",
            "Nitrogen", "Protein", "Total lipid (fat)", "Ash", "Carbohydrates",
            "Carbohydrate, by difference", "Total dietary fiber (AOAC 2011.25)",
            "High Molecular Weight Dietary Fiber (HMWDF)", "Low Molecular Weight Dietary Fiber (LMWDF)",
            "Sugars, Total", "Total Sugars", "Sucrose", "Glucose", "Fructose", "Lactose", "Maltose"
        ]
        
        # Get micronutrients (vitamins and minerals) - top 3 by value
        filtered_micronutrients = [
            item for item in nutrients 
            if item['nutrientName'] not in exclude_nutrients and item['value'] > 0
        ]
        
        # Sort by value in descending order and take top 3
        sorted_micronutrients = sorted(filtered_micronutrients, key=lambda x: x['value'], reverse=True)
        top_3_micronutrients = sorted_micronutrients[:3]
        
        # Extract only the nutrient names
        micronutrients = [item['nutrientName'] for item in top_3_micronutrients]
        result['micronutrients'] = micronutrients
        
        return result

    # Make the API call with prioritized search strategy
    try:
        query_lower = query.lower()
        
        # Step 1: Search Foundation for exact match
        print(f"🔍 Step 1: Searching Foundation for exact match: '{query}'")
        data = search_with_params("Foundation")
        if data and 'foods' in data and data['foods']:
            exact_match = find_exact_match(data['foods'], query_lower)
            if exact_match:
                print(f"✅ Found exact match in Foundation: {exact_match['description']}")
                return extract_nutrition_info(exact_match, "Foundation_exact")
        
        # Step 2: Search Foundation and take first result
        print(f"🔍 Step 2: Taking first Foundation result: '{query}'")
        if data and 'foods' in data and data['foods']:
            first_result = data['foods'][0]
            print(f"📄 Using first Foundation result: {first_result['description']}")
            return extract_nutrition_info(first_result, "Foundation_first")
        
        # Step 3: Search Survey (FNDDS) for exact match
        print(f"🔍 Step 3: Searching Survey (FNDDS) for exact match: '{query}'")
        data = search_with_params("Survey (FNDDS)")
        if data and 'foods' in data and data['foods']:
            exact_match = find_exact_match(data['foods'], query_lower)
            if exact_match:
                print(f"✅ Found exact match in Survey: {exact_match['description']}")
                return extract_nutrition_info(exact_match, "Survey_exact")
            
            # Step 4: Take first Survey result as fallback
            print(f"🔍 Step 4: Taking first Survey (FNDDS) result: '{query}'")
            first_result = data['foods'][0]
            print(f"📄 Using first Survey result: {first_result['description']}")
            return extract_nutrition_info(first_result, "Survey_first")
        
        # If no results found at all
        print(f"❌ No nutrition data found for '{query}'")
        return {
            'ingredient_name': query,
            'found_description': None,
            'search_method': 'not_found',
            'energy_kcal': None,
            'carbohydrate_g': None,
            'protein_g': None,
            'fat_g': None,
            'micronutrients': [],
            'status': 'failed',
            'error': 'No results found in any database'
        }
        
    except Exception as e:
        print(f"❌ Error processing '{query}': {e}")
        return {
            'ingredient_name': query,
            'found_description': None,
            'search_method': 'error',
            'energy_kcal': None,
            'carbohydrate_g': None,
            'protein_g': None,
            'fat_g': None,
            'micronutrients': [],
            'status': 'failed',
            'error': str(e)
        }



In [291]:
import time
from tqdm import tqdm

def process_all_ingredients(unique_ingredients_df, api_key, delay=0.1):
    """
    Process all unique ingredients to get their nutrition data.
    
    Args:
        unique_ingredients_df: DataFrame with 'ingredient_name' column
        api_key: USDA API key
        delay: Delay between API calls in seconds (to respect rate limits)
    
    Returns:
        tuple: (nutrition_df, failed_ingredients_list)
    """
    nutrition_results = []
    failed_ingredients = []
    
    print(f"Processing {len(unique_ingredients_df)} unique ingredients...")
    
    # Process each ingredient
    for idx, row in tqdm(unique_ingredients_df.iterrows(), total=len(unique_ingredients_df)):
        ingredient_name = row['ingredient_name']
        # frequency = row['frequency']
        
        try:
            # Get nutrition data
            nutrition_data = get_nutrition_data(ingredient_name, api_key)
            # nutrition_data['frequency'] = frequency
            
            if nutrition_data['status'] == 'success':
                nutrition_results.append(nutrition_data)
                print(f"✅ {ingredient_name}: Found - {nutrition_data['found_description']}")
            else:
                failed_ingredients.append({
                    'ingredient_name': ingredient_name,
                    # 'frequency': frequency,
                    'reason': nutrition_data['status']
                })
                print(f"❌ {ingredient_name}: {nutrition_data['status']}")
            
            # Add delay to respect API rate limits
            time.sleep(delay)
            
        except Exception as e:
            failed_ingredients.append({
                'ingredient_name': ingredient_name,
                # 'frequency': frequency,
                'reason': f'exception: {str(e)}'
            })
            print(f"❌ {ingredient_name}: Exception - {str(e)}")
    
    # Create DataFrames
    nutrition_df = pd.DataFrame(nutrition_results)
    failed_df = pd.DataFrame(failed_ingredients)
    
    return nutrition_df, failed_df

In [292]:
# Process all unique ingredients (excluding water)
print("Starting nutrition data collection for all unique ingredients...")
print(f"Total ingredients to process: {len(unique_ingredients_with_counts)}")

# Exclude 'water' from processing (since water has no significant nutrition and causes matching issues)
ingredients_to_exclude = ['water', 'plain water', 'boiling water', 'cold water', 'warm water']
filtered_ingredients = unique_ingredients_with_counts[
    ~unique_ingredients_with_counts['ingredient_name'].str.lower().isin([x.lower() for x in ingredients_to_exclude])
].copy()


Starting nutrition data collection for all unique ingredients...
Total ingredients to process: 873


In [293]:

# print(f"Processing ALL {len(filtered_ingredients)} ingredients (excluding water)...")
# all_nutrition_df, all_failed_df = process_all_ingredients(filtered_ingredients, api_key, delay=0.2)

# print(f"\n=== ALL INGREDIENTS PROCESSING COMPLETE ===")
# print(f"Successfully processed: {len(all_nutrition_df)} ingredients")
# print(f"Failed to process: {len(all_failed_df)} ingredients")
# print(f"Success rate: {len(all_nutrition_df) / len(filtered_ingredients) * 100:.1f}%")

# print("Note: Processing all ingredients may take a long time due to API rate limits.")

In [294]:
# # Display the nutrition DataFrame in table format
# if len(nutrition_df) > 0:
#     print("=== NUTRITION DATA SUMMARY ===")
#     print(f"Total ingredients with nutrition data: {len(nutrition_df)}")
    
#     # Create a clean nutrition DataFrame with the requested columns
#     final_nutrition_df = nutrition_df[['ingredient_name', 'found_description', 'search_method',
#                                      'energy_kcal', 'carbohydrate_g', 'protein_g', 'fat_g', 
#                                      'micronutrients']].copy()
    
#     # Create a formatted table for display
#     print("\n" + "="*140)
#     print("NUTRITION DATA TABLE - WITH PRIORITIZED SEARCH STRATEGY")
#     print("="*140)
    
#     # Create display dataframe with formatted micronutrients
#     display_df = final_nutrition_df.copy()
#     display_df['micronutrients_display'] = display_df['micronutrients'].apply(
#         lambda x: ', '.join(x[:3]) if x and len(x) > 0 else 'No data'
#     )
    
#     # Format the table headers
#     headers = ['Ingredient', 'Found As', 'Energy\n(kcal)', 'Carbs\n(g)', 'Protein\n(g)', 'Fat\n(g)', 'Top 3 Micronutrients']
    
#     # Print header
#     print(f"{'Ingredient':<15} {'Found As':<25} {'Search Method':<18} {'Energy':<8} {'Carbs':<8} {'Protein':<8} {'Fat':<8} {'Top 3 Micronutrients':<30}")
#     print(f"{'(Input)':<15} {'(USDA Database)':<25} {'(Strategy)':<18} {'(kcal)':<8} {'(g)':<8} {'(g)':<8} {'(g)':<8}")
#     print("-" * 140)
    
#     # Print data rows
#     for idx, row in display_df.iterrows():
#         ingredient = str(row['ingredient_name'])[:14]
#         found_as = str(row['found_description'])[:24] if row['found_description'] else 'N/A'
#         search_method = str(row['search_method'])[:17] if row['search_method'] else 'N/A'
#         energy = f"{row['energy_kcal']:.1f}" if pd.notna(row['energy_kcal']) else 'N/A'
#         carbs = f"{row['carbohydrate_g']:.1f}" if pd.notna(row['carbohydrate_g']) else 'N/A'
#         protein = f"{row['protein_g']:.1f}" if pd.notna(row['protein_g']) else 'N/A'
#         fat = f"{row['fat_g']:.1f}" if pd.notna(row['fat_g']) else 'N/A'
#         micros = str(row['micronutrients_display'])[:29]
        
#         print(f"{ingredient:<15} {found_as:<25} {search_method:<18} {energy:<8} {carbs:<8} {protein:<8} {fat:<8} {micros:<30}")
    
#     print("-" * 140)
    
#     # Summary statistics table
#     print(f"\n{'COLUMN COMPLETENESS SUMMARY':<40}")
#     print("-" * 50)
#     print(f"{'Column':<20} {'Complete Data':<15} {'Percentage':<15}")
#     print("-" * 50)
#     total_rows = len(final_nutrition_df)
    
#     columns_info = [
#         ('Energy (kcal)', final_nutrition_df['energy_kcal'].notna().sum()),
#         ('Carbohydrate (g)', final_nutrition_df['carbohydrate_g'].notna().sum()),
#         ('Protein (g)', final_nutrition_df['protein_g'].notna().sum()),
#         ('Fat (g)', final_nutrition_df['fat_g'].notna().sum()),
#         ('Micronutrients', (final_nutrition_df['micronutrients'].str.len() > 0).sum())
#     ]
    
#     for col_name, complete_count in columns_info:
#         percentage = (complete_count / total_rows) * 100
#         print(f"{col_name:<20} {complete_count}/{total_rows:<10} {percentage:.1f}%")
    
#     print("-" * 50)
    
#     # Display the actual DataFrame for further use
#     print(f"\nActual DataFrame (for programming use):")
#     display(final_nutrition_df)
    
# else:
#     print("No nutrition data was successfully retrieved.")

In [295]:
# # Display failed ingredients
# if len(failed_df) > 0:
#     print("=== FAILED INGREDIENTS ===")
#     print(f"Total ingredients that could not be processed: {len(failed_df)}")
    
#     # Group by failure reason
#     failure_reasons = failed_df['reason'].value_counts()
#     print(f"\nFailure reasons:")
#     for reason, count in failure_reasons.items():
#         print(f"  {reason}: {count} ingredients")
    
#     print(f"\nList of failed ingredients:")
#     for idx, row in failed_df.iterrows():
#         print(f"  - {row['ingredient_name']}  - Reason: {row['reason']}")
    
#     # Show the failed ingredients DataFrame
#     print(f"\nFailed ingredients DataFrame:")
#     print(failed_df)
# else:
#     print("🎉 All ingredients were successfully processed!")

In [296]:
# # Save the results to files
# if len(nutrition_df) > 0:
#     # Save nutrition data
#     nutrition_filename = f"../ingredient_nutrition_data_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
#     final_nutrition_df.to_excel(nutrition_filename, index=False)
#     print(f"✅ Nutrition data saved to: {nutrition_filename}")

# if len(failed_df) > 0:
#     # Save failed ingredients
#     failed_filename = f"../failed_ingredients_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
#     failed_df.to_excel(failed_filename, index=False)
#     print(f"⚠️ Failed ingredients saved to: {failed_filename}")

# print(f"\n=== SUMMARY ===")
# print(f"✅ Successfully processed: {len(nutrition_df)} ingredients")
# print(f"❌ Failed to process: {len(failed_df)} ingredients")
# print(f"📊 Success rate: {len(nutrition_df) / (len(nutrition_df) + len(failed_df)) * 100:.1f}%")

# # Show the final nutrition DataFrame structure
# if len(nutrition_df) > 0:
#     print(f"\n=== FINAL NUTRITION DATAFRAME ===")
#     print(f"Shape: {final_nutrition_df.shape}")
#     print(f"Columns: {list(final_nutrition_df.columns)}")
#     final_nutrition_df

In [297]:
# # BATCH PROCESSING: Process ingredients in batches of 150
# import math

# def process_ingredients_in_batches(ingredients_df, api_key, batch_size=150, delay=0.2, start_batch=1):
#     """
#     Process ingredients in batches to manage API rate limits and monitor progress.
    
#     Args:
#         ingredients_df: DataFrame with ingredient data
#         api_key: USDA API key
#         batch_size: Number of ingredients per batch (default: 150)
#         delay: Delay between API calls in seconds
#         start_batch: Which batch to start from (useful for resuming)
    
#     Returns:
#         tuple: (combined_nutrition_df, combined_failed_df, batch_results)
#     """
#     total_ingredients = len(ingredients_df)
#     total_batches = math.ceil(total_ingredients / batch_size)
    
#     print(f"📊 BATCH PROCESSING SETUP:")
#     print(f"   Total ingredients: {total_ingredients}")
#     print(f"   Batch size: {batch_size}")
#     print(f"   Total batches: {total_batches}")
#     print(f"   Starting from batch: {start_batch}")
#     print("=" * 60)
    
#     # Store results from all batches
#     all_nutrition_results = []
#     all_failed_results = []
#     batch_summaries = []
    
#     for batch_num in range(start_batch, total_batches + 1):
#         print(f"\n🔄 PROCESSING BATCH {batch_num}/{total_batches}")
#         print("-" * 40)
        
#         # Calculate batch indices
#         start_idx = (batch_num - 1) * batch_size
#         end_idx = min(start_idx + batch_size, total_ingredients)
        
#         # Get batch data
#         batch_data = ingredients_df.iloc[start_idx:end_idx].copy()
#         batch_ingredients = len(batch_data)
        
#         print(f"Batch {batch_num}: Processing ingredients {start_idx + 1} to {end_idx} ({batch_ingredients} ingredients)")
        
#         try:
#             # Process this batch
#             batch_nutrition_df, batch_failed_df = process_all_ingredients(
#                 batch_data, api_key, delay=delay
#             )
            
#             # Store results
#             if len(batch_nutrition_df) > 0:
#                 all_nutrition_results.append(batch_nutrition_df)
#             if len(batch_failed_df) > 0:
#                 all_failed_results.append(batch_failed_df)
            
#             # Calculate batch statistics
#             batch_success_rate = len(batch_nutrition_df) / batch_ingredients * 100
#             batch_summary = {
#                 'batch_num': batch_num,
#                 'ingredients_processed': batch_ingredients,
#                 'successful': len(batch_nutrition_df),
#                 'failed': len(batch_failed_df),
#                 'success_rate': batch_success_rate
#             }
#             batch_summaries.append(batch_summary)
            
#             print(f"\n✅ BATCH {batch_num} COMPLETE:")
#             print(f"   Successful: {len(batch_nutrition_df)}/{batch_ingredients} ({batch_success_rate:.1f}%)")
#             print(f"   Failed: {len(batch_failed_df)}")
            
#             # Save intermediate results (optional)
#             if len(batch_nutrition_df) > 0:
#                 batch_filename = f"../batch_{batch_num}_nutrition_data.xlsx"
#                 batch_nutrition_df.to_excel(batch_filename, index=False)
#                 print(f"   💾 Batch results saved: {batch_filename}")
            
#         except Exception as e:
#             print(f"❌ ERROR in batch {batch_num}: {e}")
#             batch_summary = {
#                 'batch_num': batch_num,
#                 'ingredients_processed': batch_ingredients,
#                 'successful': 0,
#                 'failed': batch_ingredients,
#                 'success_rate': 0.0,
#                 'error': str(e)
#             }
#             batch_summaries.append(batch_summary)
    
#     # Combine all results
#     print(f"\n🔗 COMBINING ALL BATCH RESULTS...")
    
#     # Combine nutrition data
#     if all_nutrition_results:
#         combined_nutrition_df = pd.concat(all_nutrition_results, ignore_index=True)
#     else:
#         combined_nutrition_df = pd.DataFrame()
    
#     # Combine failed data
#     if all_failed_results:
#         combined_failed_df = pd.concat(all_failed_results, ignore_index=True)
#     else:
#         combined_failed_df = pd.DataFrame()
    
#     return combined_nutrition_df, combined_failed_df, batch_summaries

# # Example: Process ALL ingredients in batches of 150
# print("🚀 STARTING BATCH PROCESSING FOR ALL INGREDIENTS")
# print(f"Total ingredients to process: {len(filtered_ingredients)} (water excluded)")
# print()

# # You can modify these parameters:
# BATCH_SIZE = 150       
# START_BATCH = 1           
# API_DELAY = 0.2           

# # Uncomment the lines below to start batch processing:
# all_nutrition_df, all_failed_df, batch_results = process_ingredients_in_batches(
#     filtered_ingredients, 
#     api_key, 
#     batch_size=BATCH_SIZE, 
#     delay=API_DELAY,
#     start_batch=START_BATCH
# )



In [298]:
# # Display batch processing results and provide resume functionality
# def display_batch_summary(batch_results):
#     """Display a summary of batch processing results"""
#     if not batch_results:
#         print("No batch results to display.")
#         return
    
#     print("\n" + "="*80)
#     print("📊 BATCH PROCESSING SUMMARY")
#     print("="*80)
    
#     # Create summary table
#     print(f"{'Batch':<8} {'Ingredients':<12} {'Successful':<12} {'Failed':<8} {'Success Rate':<12} {'Status':<10}")
#     print("-" * 80)
    
#     total_processed = 0
#     total_successful = 0
#     total_failed = 0
    
#     for result in batch_results:
#         batch_num = result['batch_num']
#         ingredients = result['ingredients_processed']
#         successful = result['successful']
#         # failed = result['failed']
#         success_rate = result['success_rate']
#         status = "ERROR" if 'error' in result else "COMPLETE"
        
#         print(f"{batch_num:<8} {ingredients:<12} {successful:<12} {failed:<8} {success_rate:<11.1f}% {status:<10}")
        
#         total_processed += ingredients
#         total_successful += successful
#         total_failed += failed
    
#     print("-" * 80)
#     overall_success_rate = (total_successful / total_processed * 100) if total_processed > 0 else 0
#     print(f"{'TOTAL':<8} {total_processed:<12} {total_successful:<12} {total_failed:<8} {overall_success_rate:<11.1f}% {'SUMMARY':<10}")
#     print("="*80)
    
#     return {
#         'total_processed': total_processed,
#         'total_successful': total_successful,
#         'total_failed': total_failed,
#         'overall_success_rate': overall_success_rate
#     }

# # Uncomment to display results after batch processing:
# # if 'batch_results' in locals():
# #     summary = display_batch_summary(batch_results)
# #     
# #     print(f"\n🎉 FINAL RESULTS:")
# #     print(f"   Successfully processed: {summary['total_successful']} ingredients")
# #     print(f"   Failed to process: {summary['total_failed']} ingredients") 
# #     print(f"   Overall success rate: {summary['overall_success_rate']:.1f}%")
# #     
# #     # Save final combined results
# #     if len(all_nutrition_df) > 0:
# #         final_filename = f"../final_all_ingredients_nutrition_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
# #         all_nutrition_df.to_excel(final_filename, index=False)
# #         print(f"   💾 Final nutrition data saved: {final_filename}")
# #     
# #     if len(all_failed_df) > 0:
# #         failed_filename = f"../final_failed_ingredients_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
# #         all_failed_df.to_excel(failed_filename, index=False)
# #         print(f"   💾 Failed ingredients saved: {failed_filename}")

# print("💡 RESUME FUNCTIONALITY:")
# print("   If processing stops, you can resume from any batch by changing START_BATCH parameter")
# print("   Example: START_BATCH = 3  # Resume from batch 3")
# print("   Individual batch files are saved automatically for backup")

In [299]:


# print("⚠️  TO START BATCH PROCESSING:")
# print("   1. Uncomment the lines above")
# print("   2. Run this cell")
# print("   3. Wait for completion (~25 minutes for all 828 ingredients)")
# print()
# print(f"📊 PROCESSING PLAN:")
# print(f"   • Total ingredients: {len(filtered_ingredients)} (water excluded)")
# print(f"   • Batch size: 150 ingredients")
# print(f"   • Total batches: {math.ceil(len(filtered_ingredients) / 150)}")
# print(f"   • Estimated time: ~{len(filtered_ingredients) * 0.2 / 60:.1f} minutes")
# print()
# print("💡 FEATURES:")
# print("   • Automatic progress tracking")
# print("   • Individual batch file saves (for backup)")
# print("   • Resume functionality if interrupted")
# print("   • Final combined results file")

In [300]:
# ===== DISPLAY RESULTS =====
# summary = display_batch_summary(batch_results)
# 
# print(f"\n🎉 FINAL RESULTS:")
# print(f"   Successfully processed: {summary['total_successful']} ingredients")
# print(f"   Failed to process: {summary['total_failed']} ingredients") 
# print(f"   Overall success rate: {summary['overall_success_rate']:.1f}%")
# 
# # Save final combined results
# if len(all_nutrition_df) > 0:
#     final_filename = f"../final_all_ingredients_nutrition_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
#     all_nutrition_df.to_excel(final_filename, index=False)
#     print(f"   💾 Final nutrition data saved: {final_filename}")


In [301]:
# Create 6 separate DataFrames for each batch of 150 ingredients
import math

def create_batch_dataframes(unique_ingredients_list, batch_size=150):
    """
    Create 6 separate DataFrames, each containing a batch of ingredients.
    Each DataFrame will be processed separately and can store its own nutrition data.
    
    Args:
        unique_ingredients_list: List of unique ingredients
        batch_size: Number of ingredients per batch (default: 150)
    
    Returns:
        Dictionary of DataFrames (batch_1 through batch_6)
    """
    # Calculate total number of batches needed
    total_batches = math.ceil(len(unique_ingredients_list) / batch_size)
    max_batches = 6  # Limit to 6 batches as requested
    
    print(f"Total ingredients: {len(unique_ingredients_list)}")
    print(f"Batch size: {batch_size}")
    print(f"Total batches needed: {total_batches}")
    print(f"Creating {min(total_batches, max_batches)} DataFrames")
    print("=" * 50)
    
    batch_dataframes = {}
    
    for batch_num in range(1, min(total_batches, max_batches) + 1):
        # Calculate start and end indices for this batch
        start_idx = (batch_num - 1) * batch_size
        end_idx = min(start_idx + batch_size, len(unique_ingredients_list))
        
        # Get ingredients for this batch
        batch_ingredients = unique_ingredients_list[start_idx:end_idx]
        
        # Create DataFrame for this batch with empty nutrition columns
        batch_df = pd.DataFrame({
            'ingredient': batch_ingredients,
            'description': [None] * len(batch_ingredients),
            'energy_kcal_per_100g': [None] * len(batch_ingredients),
            'carbs_g_per_100g': [None] * len(batch_ingredients),
            'protein_g_per_100g': [None] * len(batch_ingredients),
            'fat_g_per_100g': [None] * len(batch_ingredients),
            'top_micronutrients': [None] * len(batch_ingredients),
            'search_method': [None] * len(batch_ingredients),
            'batch_number': [batch_num] * len(batch_ingredients)
        })
        
        # Store in dictionary
        batch_dataframes[f'batch_{batch_num}'] = batch_df
        
        print(f"Batch {batch_num}: {len(batch_ingredients)} ingredients (indices {start_idx}-{end_idx-1})")
        print(f"  Sample ingredients: {batch_ingredients[:3]}...")
        print()
    
    return batch_dataframes



In [302]:
# Create the unique ingredients list from filtered_ingredients DataFrame
unique_ingredients_list = filtered_ingredients['ingredient_name'].tolist()
print(f"Extracted {len(unique_ingredients_list)} unique ingredients from filtered_ingredients DataFrame")

# Create the batch DataFrames
print("Creating 6 separate DataFrames for batch processing...")
batch_dfs = create_batch_dataframes(unique_ingredients_list, batch_size=150)

# Display information about each batch
print("\n📊 BATCH DATAFRAMES CREATED:")
print("=" * 60)
for batch_name, df in batch_dfs.items():
    print(f"{batch_name.upper()}:")
    print(f"  - Shape: {df.shape}")
    print(f"  - Ingredient range: {df['ingredient'].iloc[0]} ... {df['ingredient'].iloc[-1]}")
    print(f"  - Batch number: {df['batch_number'].iloc[0]}")
    print()

Extracted 870 unique ingredients from filtered_ingredients DataFrame
Creating 6 separate DataFrames for batch processing...
Total ingredients: 870
Batch size: 150
Total batches needed: 6
Creating 6 DataFrames
Batch 1: 150 ingredients (indices 0-149)
  Sample ingredients: ['vegetable oil', 'onion', 'olive oil']...

Batch 2: 150 ingredients (indices 150-299)
  Sample ingredients: ['congee or ½ bowl soft rice', 'melted cheese', 'breast/formula milk']...

Batch 3: 150 ingredients (indices 300-449)
  Sample ingredients: ['cooked mashed pumpkin', 'dried dates (optional)', 'potato (peeled) (50g)']...

Batch 4: 150 ingredients (indices 450-599)
  Sample ingredients: ['beef mince', 'Carrot', 'chicken broth']...

Batch 5: 150 ingredients (indices 600-749)
  Sample ingredients: ['knob of butter', '(4 oz/125 g) fresh or thawed frozen raspberries', 'white']...

Batch 6: 120 ingredients (indices 750-869)
  Sample ingredients: ['new potatoes (1 portion)', 'white bread', 'turmeric']...


📊 BATCH DATAF

In [303]:
# 🔧 IMPORT FIX - Run this cell before batch processing
import datetime
import time
print("✅ All required imports loaded successfully!")
print("📅 Current timestamp:", datetime.datetime.now().strftime('%Y%m%d_%H%M%S'))
print("🚀 Ready to start batch processing!")

✅ All required imports loaded successfully!
📅 Current timestamp: 20250624_104309
🚀 Ready to start batch processing!


In [304]:
# Function to process individual batch DataFrames
import datetime  # Add missing import

def process_batch_dataframe(batch_df, batch_name, api_key, save_results=True):
    """
    Process a single batch DataFrame by fetching nutrition data for all ingredients.
    
    Args:
        batch_df: DataFrame containing ingredients for this batch
        batch_name: Name of the batch (e.g., 'batch_1')
        api_key: USDA API key
        save_results: Whether to save results to Excel file
    
    Returns:
        Tuple of (processed_df, failed_ingredients_list)
    """
    print(f"\n🔄 PROCESSING {batch_name.upper()}")
    print("=" * 50)
    
    processed_df = batch_df.copy()
    failed_ingredients = []
    successful_count = 0
    
    total_ingredients = len(batch_df)
    
    for idx, row in batch_df.iterrows():
        ingredient = row['ingredient']
        
        # Progress indicator
        current_position = idx - batch_df.index[0] + 1
        print(f"Processing {current_position}/{total_ingredients}: {ingredient}")
        
        # Fetch nutrition data
        nutrition_data = get_nutrition_data(ingredient, api_key)
        
        if nutrition_data and nutrition_data['status'] == 'success':
            # Update the DataFrame with nutrition data
            processed_df.at[idx, 'description'] = nutrition_data.get('found_description', ingredient)
            processed_df.at[idx, 'energy_kcal_per_100g'] = nutrition_data.get('energy_kcal')
            processed_df.at[idx, 'energy_kcal_per_100g'] = nutrition_data.get('energy_kcal')
            processed_df.at[idx, 'carbs_g_per_100g'] = nutrition_data.get('carbohydrate_g')
            processed_df.at[idx, 'protein_g_per_100g'] = nutrition_data.get('protein_g')
            processed_df.at[idx, 'fat_g_per_100g'] = nutrition_data.get('fat_g')
            processed_df.at[idx, 'top_micronutrients'] = ', '.join(nutrition_data.get('micronutrients', []))
            processed_df.at[idx, 'search_method'] = nutrition_data.get('search_method')
            
            successful_count += 1
            print(f"  ✅ Success - Method: {nutrition_data.get('search_method')}")
        else:
            failed_ingredients.append(ingredient)
            print(f"  ❌ Failed")
        
        # Small delay to avoid overwhelming the API
        time.sleep(0.1)
    
    # Summary
    print(f"\n📊 {batch_name.upper()} SUMMARY:")
    print(f"  - Total ingredients: {total_ingredients}")
    print(f"  - Successful: {successful_count}")
    print(f"  - Failed: {len(failed_ingredients)}")
    print(f"  - Success rate: {successful_count/total_ingredients*100:.1f}%")
    
    # if save_results:
    #     # Save processed DataFrame
    #     results_filename = f"nutrition_results_{batch_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    #     processed_df.to_excel(results_filename, index=False)
    #     print(f"  💾 Results saved to: {results_filename}")
        
    #     # Save failed ingredients
    #     if failed_ingredients:
    #         failed_filename = f"failed_ingredients_{batch_name}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    #         failed_df = pd.DataFrame({'failed_ingredient': failed_ingredients, 'batch': batch_name})
    #         failed_df.to_excel(failed_filename, index=False)
    #         print(f"  💾 Failed ingredients saved to: {failed_filename}")
    
    return processed_df, failed_ingredients

In [305]:
# Function to process all batches or specific batches
def process_selected_batches(batch_dfs, api_key, batch_numbers=None, save_results=True):

    processed_results = {}
    all_failed = {}
    
    # Determine which batches to process
    if batch_numbers is None:
        batches_to_process = list(batch_dfs.keys())
    else:
        batches_to_process = [f'batch_{num}' for num in batch_numbers if f'batch_{num}' in batch_dfs]
    
    print(f"🚀 STARTING BATCH PROCESSING")
    print(f"Batches to process: {batches_to_process}")
    print("=" * 60)
    
    for batch_name in batches_to_process:
        if batch_name in batch_dfs:
            try:
                processed_df, failed_list = process_batch_dataframe(
                    batch_dfs[batch_name], 
                    batch_name, 
                    api_key, 
                    save_results
                )
                processed_results[batch_name] = processed_df
                all_failed[batch_name] = failed_list
                
            except Exception as e:
                print(f"❌ Error processing {batch_name}: {str(e)}")
                all_failed[batch_name] = list(batch_dfs[batch_name]['ingredient'])
        
        print("\n" + "="*60)
    
    # Overall summary
    total_ingredients = sum(len(df) for df in batch_dfs.values() if any(batch in batch_dfs for batch in batches_to_process))
    total_successful = sum(len(df[df['search_method'].notna()]) for df in processed_results.values())
    total_failed = sum(len(failed_list) for failed_list in all_failed.values())
    
    print(f"\n🎯 OVERALL PROCESSING SUMMARY:")
    print(f"  - Total ingredients processed: {total_successful + total_failed}")
    print(f"  - Successful: {total_successful}")
    print(f"  - Failed: {total_failed}")
    print(f"  - Overall success rate: {total_successful/(total_successful + total_failed)*100:.1f}%")
    
    return processed_results, all_failed

In [306]:
# 📊 BATCH 1 PROCESSING (Ingredients 1-150)
print("🔄 PROCESSING BATCH 1")
print("=" * 50)
print(f"Batch 1 contains {len(batch_dfs['batch_1'])} ingredients")
print(f"Sample ingredients: {list(batch_dfs['batch_1']['ingredient'].head())}")
print()

# Process Batch 1
batch_1_results, batch_1_failed = process_batch_dataframe(
    batch_dfs['batch_1'], 
    'batch_1', 
    api_key, 
    save_results=True
)

print(f"\n✅ BATCH 1 COMPLETED!")


🔄 PROCESSING BATCH 1
Batch 1 contains 150 ingredients
Sample ingredients: ['vegetable oil', 'onion', 'olive oil', 'butter', 'carrot']


🔄 PROCESSING BATCH_1
Processing 1/150: vegetable oil
🔍 Step 1: Searching Foundation for exact match: 'vegetable oil'
🔍 Step 2: Taking first Foundation result: 'vegetable oil'
📄 Using first Foundation result: Oil, canola
  ✅ Success - Method: Foundation_first
Processing 2/150: onion
🔍 Step 1: Searching Foundation for exact match: 'onion'
✅ Found exact match in Foundation: Green onion, (scallion), bulb and greens, root removed, raw
  ✅ Success - Method: Foundation_exact
Processing 3/150: olive oil
🔍 Step 1: Searching Foundation for exact match: 'olive oil'
🔍 Step 2: Taking first Foundation result: 'olive oil'
📄 Using first Foundation result: Oil, olive, extra light
  ✅ Success - Method: Foundation_first
Processing 4/150: butter
🔍 Step 1: Searching Foundation for exact match: 'butter'
✅ Found exact match in Foundation: Almond butter, creamy
  ✅ Success - 

In [307]:
# 🔍 CHECK BATCH 1 RESULTS
print("📋 BATCH 1 DETAILED RESULTS")
print("=" * 50)

if 'batch_1_results' in locals():
    print(f"✅ Successfully processed: {len(batch_1_results[batch_1_results['search_method'].notna()])} ingredients")
    print(f"❌ Failed: {len(batch_1_failed)} ingredients")
    print(f"📊 Success rate: {len(batch_1_results[batch_1_results['search_method'].notna()]) / len(batch_1_results) * 100:.1f}%")
    
    # Show full results DataFrame
    print(f"\n📊 FULL BATCH 1 RESULTS:")
    display(batch_1_results)
    
    # Show failed ingredients if any
    if batch_1_failed:
        print(f"\n❌ FAILED INGREDIENTS IN BATCH 1:")
        for ingredient in batch_1_failed:
            print(f"  - {ingredient}")
    
    print(f"\n✅ Ready to proceed to Batch 2? Run the next cell!")
else:
    print("⚠️ Batch 1 not processed yet. Run the previous cell first.")

📋 BATCH 1 DETAILED RESULTS
✅ Successfully processed: 127 ingredients
❌ Failed: 23 ingredients
📊 Success rate: 84.7%

📊 FULL BATCH 1 RESULTS:


,ingredient,description,energy_kcal_per_100g,carbs_g_per_100g,protein_g_per_100g,fat_g_per_100g,top_micronutrients,search_method,batch_number
0,vegetable oil,"Oil, canola",None,None,None,None,"Beta-sitosterol, Campesterol, Total fat (NLEA)",Foundation_first,1
1,onion,"Green onion, (scallion), bulb and greens, root...",None,None,0.669,None,"Potassium, K, Calcium, Ca, Phosphorus, P",Foundation_exact,1
2,olive oil,"Oil, olive, extra light",None,None,None,None,"Beta-sitosterol, Total fat (NLEA), Fatty acids...",Foundation_first,1
3,butter,"Almond butter, creamy",603,21.2,20.8,53.0,"Potassium, K, Phosphorus, P, Magnesium, Mg",Foundation_exact,1
4,carrot,"Carrots, baby, raw",38.3,9.08,0.805,0.138,"Potassium, K, Sodium, Na, Folate, total",Foundation_first,1
...,...,...,...,...,...,...,...,...,...
145,grated cheese,"Cheese, parmesan, grated",None,12.4,29.6,28.0,"Sodium, Na, Calcium, Ca, Phosphorus, P",Foundation_first,1
146,chopped parsley,"Pork, chop, center cut, raw",145,-0.562,22.8,5.48,"Potassium, K, Phosphorus, P, Cholesterol",Foundation_first,1
147,tofu,"Soup, miso or tofu",None,2.3,2.21,0.85,"Sodium, Na, Lutein + zeaxanthin, Potassium, K",Survey_exact,1
148,coconut oil,"Oil, coconut",None,0.84,0.0,99.1,"Total fat (NLEA), Fatty acids, total saturated...",Foundation_first,1



❌ FAILED INGREDIENTS IN BATCH 1:
  - crème fraîche
  - full-fat milk
  - clove
  - salmon fillet
  - milk_x000D_
  - slice wholemeal bread (1 portion)
  - slice wholemeal bread
  - plain full-fat yoghurt
  - full-fat hard cheese
  - thyme
  - breadcrumbs
  - cornstarch
  - paprika
  - courgette
  - cod fillet
  - full fat hard cheese
  - self-raising flour
  - plain yogurt
  - full-fat cream cheese
  - fresh parsley (1 tbsp
  - tinned sweetcorn
  - florets (35g)
  - yogurt

✅ Ready to proceed to Batch 2? Run the next cell!


In [308]:
# 📊 BATCH 2 PROCESSING (Ingredients 151-300)
print("🔄 PROCESSING BATCH 2")
print("=" * 50)
print(f"Batch 2 contains {len(batch_dfs['batch_2'])} ingredients")
print(f"Sample ingredients: {list(batch_dfs['batch_2']['ingredient'].head())}")
print()

# Process Batch 2
batch_2_results, batch_2_failed = process_batch_dataframe(
    batch_dfs['batch_2'], 
    'batch_2', 
    api_key, 
    save_results=True
)

print(f"\n✅ BATCH 2 COMPLETED!")

🔄 PROCESSING BATCH 2
Batch 2 contains 150 ingredients
Sample ingredients: ['congee or ½ bowl soft rice', 'melted cheese', 'breast/formula milk', 'pitta bread', 'crème fraiche']


🔄 PROCESSING BATCH_2
Processing 1/150: congee or ½ bowl soft rice
🔍 Step 1: Searching Foundation for exact match: 'congee or ½ bowl soft rice'
🔍 Step 2: Taking first Foundation result: 'congee or ½ bowl soft rice'
📄 Using first Foundation result: Cookies, oatmeal, soft, with raisins
  ✅ Success - Method: Foundation_first
Processing 2/150: melted cheese
🔍 Step 1: Searching Foundation for exact match: 'melted cheese'
🔍 Step 2: Taking first Foundation result: 'melted cheese'
📄 Using first Foundation result: Cheese, cheddar
  ✅ Success - Method: Foundation_first
Processing 3/150: breast/formula milk
🔍 Step 1: Searching Foundation for exact match: 'breast/formula milk'
❌ API request failed: 500
🔍 Step 2: Taking first Foundation result: 'breast/formula milk'
🔍 Step 3: Searching Survey (FNDDS) for exact match: 'breas

In [309]:
# 🔍 CHECK BATCH 2 RESULTS
print("📋 BATCH 2 DETAILED RESULTS")
print("=" * 50)

if 'batch_2_results' in locals():
    print(f"✅ Successfully processed: {len(batch_2_results[batch_2_results['search_method'].notna()])} ingredients")
    print(f"❌ Failed: {len(batch_2_failed)} ingredients")
    print(f"📊 Success rate: {len(batch_2_results[batch_2_results['search_method'].notna()]) / len(batch_2_results) * 100:.1f}%")
    
    # Show full results DataFrame
    print(f"\n📊 FULL BATCH 2 RESULTS:")
    display(batch_2_results)
    
    # Show failed ingredients if any
    if batch_2_failed:
        print(f"\n❌ FAILED INGREDIENTS IN BATCH 2:")
        for ingredient in batch_2_failed:
            print(f"  - {ingredient}")
    
    print(f"\n✅ Ready to proceed to Batch 3? Run the next cell!")
else:
    print("⚠️ Batch 2 not processed yet. Run the previous cell first.")

📋 BATCH 2 DETAILED RESULTS
✅ Successfully processed: 118 ingredients
❌ Failed: 32 ingredients
📊 Success rate: 78.7%

📊 FULL BATCH 2 RESULTS:


,ingredient,description,energy_kcal_per_100g,carbs_g_per_100g,protein_g_per_100g,fat_g_per_100g,top_micronutrients,search_method,batch_number
0,congee or ½ bowl soft rice,"Cookies, oatmeal, soft, with raisins",None,69.6,5.79,14.3,"Sodium, Na, Potassium, K, Phosphorus, P",Foundation_first,2
1,melted cheese,"Cheese, cheddar",None,2.44,23.3,34.0,"Calcium, Ca, Sodium, Na, Phosphorus, P",Foundation_first,2
2,breast/formula milk,None,None,None,None,None,None,None,2
3,pitta bread,"Bread, white, commercially prepared",None,49.2,9.43,3.59,"Sodium, Na, Calcium, Ca, Potassium, K",Foundation_first,2
4,crème fraiche,None,None,None,None,None,None,None,2
...,...,...,...,...,...,...,...,...,...
145,(20g) dried kelp,"Egg, white, dried",None,6.02,79.9,0.65,"Sodium, Na, Potassium, K, Phosphorus, P",Foundation_first,2
146,(3.5 oz) leek white part (about 1 large leek w...,"Leeks, bulb and greens, root removed, raw",None,None,1.47,None,"Potassium, K, Calcium, Ca, Phosphorus, P",Foundation_first,2
147,carrots (peeled) (200g),"Cucumber, with peel, raw",13.9,2.95,0.625,0.178,"Potassium, K, Vitamin K (phylloquinone), Phosp...",Foundation_first,2
148,porridge oats,"Flour, oat, whole grain",386,69.9,13.2,6.31,"Potassium, K, Phosphorus, P, Magnesium, Mg",Foundation_first,2



❌ FAILED INGREDIENTS IN BATCH 2:
  - breast/formula milk
  - crème fraiche
  - beetroot
  - full fat cream cheese (1 portion)
  - spear (38g)
  - florets
  - baking potato (1/3 to serve
  - (30g)
  - courgettes
  - cod (90g)
  - (4 oz/125 g) cauliflower florets
  - water_x000D_
  - butter_x000D_
  - mascarpone
  - plain whole milk yoghurt
  - salmon fillet (1/2 to serve)
  - (35g)
  - wholemeal self-raising flour
  - tempeh
  - Patola
  - fresh cream
  - chickpeas (½ canned in water
  - Bangus
  - cowpeas
  - canned mixed beans (chickpeas
  - passata_x000D_
  - (about 10.5oz / 300g)
  - (310g)
  - (130g)
  - (10ml)
  - cooker / or 1 steam cooker
  - potato (about 1.8oz / 50g)

✅ Ready to proceed to Batch 3? Run the next cell!


In [310]:
# 📊 BATCH 3 PROCESSING (Ingredients 301-450)
print("🔄 PROCESSING BATCH 3")
print("=" * 50)
print(f"Batch 3 contains {len(batch_dfs['batch_3'])} ingredients")
print(f"Sample ingredients: {list(batch_dfs['batch_3']['ingredient'].head())}")
print()

# Process Batch 3
batch_3_results, batch_3_failed = process_batch_dataframe(
    batch_dfs['batch_3'], 
    'batch_3', 
    api_key, 
    save_results=True
)

print(f"\n✅ BATCH 3 COMPLETED!")

🔄 PROCESSING BATCH 3
Batch 3 contains 150 ingredients
Sample ingredients: ['cooked mashed pumpkin', 'dried dates (optional)', 'potato (peeled) (50g)', '(7 oz) carrot (about 2 large carrots)', 'spinach_x000D_']


🔄 PROCESSING BATCH_3
Processing 1/150: cooked mashed pumpkin
🔍 Step 1: Searching Foundation for exact match: 'cooked mashed pumpkin'
🔍 Step 2: Taking first Foundation result: 'cooked mashed pumpkin'
📄 Using first Foundation result: Seeds, pumpkin seeds (pepitas), raw
  ✅ Success - Method: Foundation_first
Processing 2/150: dried dates (optional)
🔍 Step 1: Searching Foundation for exact match: 'dried dates (optional)'
🔍 Step 2: Taking first Foundation result: 'dried dates (optional)'
📄 Using first Foundation result: Egg, white, dried
  ✅ Success - Method: Foundation_first
Processing 3/150: potato (peeled) (50g)
🔍 Step 1: Searching Foundation for exact match: 'potato (peeled) (50g)'
🔍 Step 2: Taking first Foundation result: 'potato (peeled) (50g)'
📄 Using first Foundation result:

In [311]:
# 🔍 CHECK BATCH 3 RESULTS
print("📋 BATCH 3 DETAILED RESULTS")
print("=" * 50)

if 'batch_3_results' in locals():
    print(f"✅ Successfully processed: {len(batch_3_results[batch_3_results['search_method'].notna()])} ingredients")
    print(f"❌ Failed: {len(batch_3_failed)} ingredients")
    print(f"📊 Success rate: {len(batch_3_results[batch_3_results['search_method'].notna()]) / len(batch_3_results) * 100:.1f}%")
    
    # Show full results DataFrame
    print(f"\n📊 FULL BATCH 3 RESULTS:")
    display(batch_3_results)
    
    # Show failed ingredients if any
    if batch_3_failed:
        print(f"\n❌ FAILED INGREDIENTS IN BATCH 3:")
        for ingredient in batch_3_failed:
            print(f"  - {ingredient}")
    
    print(f"\n✅ Ready to proceed to Batch 4? Run the next cell!")
else:
    print("⚠️ Batch 3 not processed yet. Run the previous cell first.")

📋 BATCH 3 DETAILED RESULTS
✅ Successfully processed: 123 ingredients
❌ Failed: 27 ingredients
📊 Success rate: 82.0%

📊 FULL BATCH 3 RESULTS:


,ingredient,description,energy_kcal_per_100g,carbs_g_per_100g,protein_g_per_100g,fat_g_per_100g,top_micronutrients,search_method,batch_number
0,cooked mashed pumpkin,"Seeds, pumpkin seeds (pepitas), raw",515,18.7,29.9,40.0,"Phosphorus, P, Potassium, K, Magnesium, Mg",Foundation_first,3
1,dried dates (optional),"Egg, white, dried",None,6.02,79.9,0.65,"Sodium, Na, Potassium, K, Phosphorus, P",Foundation_first,3
2,potato (peeled) (50g),"Cucumber, with peel, raw",13.9,2.95,0.625,0.178,"Potassium, K, Vitamin K (phylloquinone), Phosp...",Foundation_first,3
3,(7 oz) carrot (about 2 large carrots),"Carrots, baby, raw",38.3,9.08,0.805,0.138,"Potassium, K, Sodium, Na, Folate, total",Foundation_first,3
4,spinach_x000D_,None,None,None,None,None,None,None,3
...,...,...,...,...,...,...,...,...,...
145,chicken,"Chicken, ground, with additives, raw",138,0.0,17.9,7.16,"Potassium, K, Phosphorus, P, Cholesterol",Foundation_exact,3
146,salmon,None,None,None,None,None,None,None,3
147,chicken liver,"Chicken, ground, with additives, raw",138,0.0,17.9,7.16,"Potassium, K, Phosphorus, P, Cholesterol",Foundation_first,3
148,kabocha pumpkin,"Seeds, pumpkin seeds (pepitas), raw",515,18.7,29.9,40.0,"Phosphorus, P, Potassium, K, Magnesium, Mg",Foundation_first,3



❌ FAILED INGREDIENTS IN BATCH 3:
  - spinach_x000D_
  - sweet potato (about 12 oz / 350 g before peeling)
  - flour or cornstarch (optional
  - peeled carrot (sliced
  - (0.5 fl oz) heavy cream (or soy milk for dairy-free option)
  - (30 g) plain yogurt
  - yogurt (use the empty cup as a measuring unit)
  - finely grated lemon zest (about 1 g
  - (skin and seeds removed
  - (200g)
  - sweet potato (peeled weight
  - zucchini (peeled weight
  - potatoes suitable for mashing (about 8.8 oz/250g)
  - stalks
  - chicken (upper/lower thigh or wing)
  - lemongrass (bruised)
  - kangkung
  - cauliflower/broccoli
  - lemongrass
  - tempe
  - cod (no seasoning)
  - drummets
  - coconut milk (you can also use almond milk)
  - Kulitis
  - catfish fillet
  - minced shrimp
  - salmon

✅ Ready to proceed to Batch 4? Run the next cell!


In [312]:
# 📊 BATCH 4 PROCESSING (Ingredients 451-600)
print("🔄 PROCESSING BATCH 4")
print("=" * 50)
print(f"Batch 4 contains {len(batch_dfs['batch_4'])} ingredients")
print(f"Sample ingredients: {list(batch_dfs['batch_4']['ingredient'].head())}")
print()

# Process Batch 4
batch_4_results, batch_4_failed = process_batch_dataframe(
    batch_dfs['batch_4'], 
    'batch_4', 
    api_key, 
    save_results=True
)

print(f"\n✅ BATCH 4 COMPLETED!")


🔄 PROCESSING BATCH 4
Batch 4 contains 150 ingredients
Sample ingredients: ['beef mince', 'Carrot', 'chicken broth', 'chicken meat', 'Potato']


🔄 PROCESSING BATCH_4
Processing 1/150: beef mince
🔍 Step 1: Searching Foundation for exact match: 'beef mince'
🔍 Step 2: Taking first Foundation result: 'beef mince'
📄 Using first Foundation result: Frankfurter, beef, unheated
  ✅ Success - Method: Foundation_first
Processing 2/150: Carrot
🔍 Step 1: Searching Foundation for exact match: 'Carrot'
🔍 Step 2: Taking first Foundation result: 'Carrot'
📄 Using first Foundation result: Carrots, baby, raw
  ✅ Success - Method: Foundation_first
Processing 3/150: chicken broth
🔍 Step 1: Searching Foundation for exact match: 'chicken broth'
🔍 Step 2: Taking first Foundation result: 'chicken broth'
📄 Using first Foundation result: Chicken, ground, with additives, raw
  ✅ Success - Method: Foundation_first
Processing 4/150: chicken meat
🔍 Step 1: Searching Foundation for exact match: 'chicken meat'
🔍 Step 2:

In [313]:
# 🔍 CHECK BATCH 4 RESULTS
print("📋 BATCH 4 DETAILED RESULTS")
print("=" * 50)

if 'batch_4_results' in locals():
    print(f"✅ Successfully processed: {len(batch_4_results[batch_4_results['search_method'].notna()])} ingredients")
    print(f"❌ Failed: {len(batch_4_failed)} ingredients")
    print(f"📊 Success rate: {len(batch_4_results[batch_4_results['search_method'].notna()]) / len(batch_4_results) * 100:.1f}%")
    
    # Show full results DataFrame
    print(f"\n📊 FULL BATCH 4 RESULTS:")
    display(batch_4_results)
    
    # Show failed ingredients if any
    if batch_4_failed:
        print(f"\n❌ FAILED INGREDIENTS IN BATCH 4:")
        for ingredient in batch_4_failed:
            print(f"  - {ingredient}")
    
    print(f"\n✅ Ready to proceed to Batch 5? Run the next cell!")
else:
    print("⚠️ Batch 4 not processed yet. Run the previous cell first.")

📋 BATCH 4 DETAILED RESULTS
✅ Successfully processed: 116 ingredients
❌ Failed: 34 ingredients
📊 Success rate: 77.3%

📊 FULL BATCH 4 RESULTS:


,ingredient,description,energy_kcal_per_100g,carbs_g_per_100g,protein_g_per_100g,fat_g_per_100g,top_micronutrients,search_method,batch_number
0,beef mince,"Frankfurter, beef, unheated",None,2.89,11.7,28.0,"Sodium, Na, Potassium, K, Phosphorus, P",Foundation_first,4
1,Carrot,"Carrots, baby, raw",38.3,9.08,0.805,0.138,"Potassium, K, Sodium, Na, Folate, total",Foundation_first,4
2,chicken broth,"Chicken, ground, with additives, raw",138,0.0,17.9,7.16,"Potassium, K, Phosphorus, P, Cholesterol",Foundation_first,4
3,chicken meat,"Chicken, breast, meat and skin, raw",133,-0.428,21.4,4.78,"Potassium, K, Phosphorus, P, Cholesterol",Foundation_first,4
4,Potato,"Flour, potato",353,79.9,8.11,0.951,"Potassium, K, Phosphorus, P, Magnesium, Mg",Foundation_exact,4
...,...,...,...,...,...,...,...,...,...
145,chopped herbs,"Pork, chop, center cut, raw",145,-0.562,22.8,5.48,"Potassium, K, Phosphorus, P, Cholesterol",Foundation_first,4
146,chard,"Chard, cooked",None,3.9,1.88,2.95,"Lutein + zeaxanthin, Carotene, beta, Vitamin K...",Survey_exact,4
147,crumbled tuna (not in brine or oil),None,None,None,None,None,None,None,4
148,pumpkin flesh,"Sweet potatoes, orange flesh, without skin, raw",77.4,17.3,1.58,0.375,"Potassium, K, Phosphorus, P, Calcium, Ca",Foundation_first,4



❌ FAILED INGREDIENTS IN BATCH 4:
  - Yoghurt
  - penne
  - small pasta shapes (e.g. orzo
  - mascarpone _x000D_
  - panko breadcrumbs _x000D_
  - egg_x000D_
  - panko breadcrumbs  _x000D_
  - cornflour  _x000D_
  - yoghurt_x000D_
  - polenta (a coarse
  - flour_x000D_
  - blueberries_x000D_
  - risoni
  - mee sua
  - breast_x000D_
  - sweetcorn _x000D_
  - salmon (one fillet)
  - low salt vegetable stock _x000D_
  - kale_x000D_
  - onion_x000D_
  - (55g) of apple (Golden
  - self-raising flour (plus a little extra for kneading)
  - cream cheese (Philadelphia
  - (12 fl oz/375 ml) water or low-sodium vegetable stock
  - (3½ oz/105 g) risotto rice
  - cornflour
  - yoghurt
  - (8 fl oz/250 ml) water or low-sodium vegetable or chicken stock
  - (4 oz/125 g) pearl barley
  - cornflour (Maïzena®)
  - sage
  - butternut squash (about 1 lb/500 g)
  - (5 oz/155 g) frozen petite peas
  - crumbled tuna (not in brine or oil)

✅ Ready to proceed to Batch 5? Run the next cell!


In [314]:
# 📊 BATCH 5 PROCESSING (Ingredients 601-750)
print("🔄 PROCESSING BATCH 5")
print("=" * 50)
print(f"Batch 5 contains {len(batch_dfs['batch_5'])} ingredients")
print(f"Sample ingredients: {list(batch_dfs['batch_5']['ingredient'].head())}")
print()

# Process Batch 5
batch_5_results, batch_5_failed = process_batch_dataframe(
    batch_dfs['batch_5'], 
    'batch_5', 
    api_key, 
    save_results=True
)

print(f"\n✅ BATCH 5 COMPLETED!")

🔄 PROCESSING BATCH 5
Batch 5 contains 150 ingredients
Sample ingredients: ['knob of butter', '(4 oz/125 g) fresh or thawed frozen raspberries', 'white', 'granulated sugar', 'cinnamon']


🔄 PROCESSING BATCH_5
Processing 1/150: knob of butter
🔍 Step 1: Searching Foundation for exact match: 'knob of butter'


🔍 Step 2: Taking first Foundation result: 'knob of butter'
📄 Using first Foundation result: Almond butter, creamy
  ✅ Success - Method: Foundation_first
Processing 2/150: (4 oz/125 g) fresh or thawed frozen raspberries
🔍 Step 1: Searching Foundation for exact match: '(4 oz/125 g) fresh or thawed frozen raspberries'
❌ API request failed: 500
🔍 Step 2: Taking first Foundation result: '(4 oz/125 g) fresh or thawed frozen raspberries'
🔍 Step 3: Searching Survey (FNDDS) for exact match: '(4 oz/125 g) fresh or thawed frozen raspberries'
❌ API request failed: 500
❌ No nutrition data found for '(4 oz/125 g) fresh or thawed frozen raspberries'
  ❌ Failed
Processing 3/150: white
🔍 Step 1: Searching Foundation for exact match: 'white'
✅ Found exact match in Foundation: Egg, white, dried
  ✅ Success - Method: Foundation_exact
Processing 4/150: granulated sugar
🔍 Step 1: Searching Foundation for exact match: 'granulated sugar'
🔍 Step 2: Taking first Foundation result: 'granulated sugar'
📄 Using fir

In [315]:
# 🔍 CHECK BATCH 5 RESULTS
print("📋 BATCH 5 DETAILED RESULTS")
print("=" * 50)

if 'batch_5_results' in locals():
    print(f"✅ Successfully processed: {len(batch_5_results[batch_5_results['search_method'].notna()])} ingredients")
    print(f"❌ Failed: {len(batch_5_failed)} ingredients")
    print(f"📊 Success rate: {len(batch_5_results[batch_5_results['search_method'].notna()]) / len(batch_5_results) * 100:.1f}%")
    
    # Show full results DataFrame
    print(f"\n📊 FULL BATCH 5 RESULTS:")
    display(batch_5_results)
    
    # Show failed ingredients if any
    if batch_5_failed:
        print(f"\n❌ FAILED INGREDIENTS IN BATCH 5:")
        for ingredient in batch_5_failed:
            print(f"  - {ingredient}")
    
    print(f"\n✅ Ready to proceed to Batch 6 (FINAL)? Run the next cell!")
else:
    print("⚠️ Batch 5 not processed yet. Run the previous cell first.")

📋 BATCH 5 DETAILED RESULTS
✅ Successfully processed: 108 ingredients
❌ Failed: 42 ingredients
📊 Success rate: 72.0%

📊 FULL BATCH 5 RESULTS:


,ingredient,description,energy_kcal_per_100g,carbs_g_per_100g,protein_g_per_100g,fat_g_per_100g,top_micronutrients,search_method,batch_number
0,knob of butter,"Almond butter, creamy",603,21.2,20.8,53.0,"Potassium, K, Phosphorus, P, Magnesium, Mg",Foundation_first,5
1,(4 oz/125 g) fresh or thawed frozen raspberries,None,None,None,None,None,None,None,5
2,white,"Egg, white, dried",None,6.02,79.9,0.65,"Sodium, Na, Potassium, K, Phosphorus, P",Foundation_exact,5
3,granulated sugar,"Sugars, granulated",None,99.6,0.0,0.32,"Potassium, K, Sodium, Na, Calcium, Ca",Foundation_first,5
4,cinnamon,"Bread, cinnamon",None,44.38,7.05,5.29,"Sodium, Na, Folate, DFE, Potassium, K",Survey_exact,5
...,...,...,...,...,...,...,...,...,...
145,salt-reduced vegetable stock or water,"Butter, stick, salted",None,None,None,82.2,"Vitamin A, RAE, Retinol, Sodium, Na",Foundation_first,5
146,chopped canned tomatoes,"Tomato, puree, canned",34.8,8.04,1.58,0.265,"Potassium, K, Phosphorus, P, Sodium, Na",Foundation_first,5
147,cucumber finger sticks,"Cucumber, with peel, raw",13.9,2.95,0.625,0.178,"Potassium, K, Vitamin K (phylloquinone), Phosp...",Foundation_first,5
148,salmon fillet (½ to serve),"Sauce, salsa, ready-to-serve",None,6.74,1.44,0.19,"Lycopene, Sodium, Na, Carotene, beta",Foundation_first,5



❌ FAILED INGREDIENTS IN BATCH 5:
  - (4 oz/125 g) fresh or thawed frozen raspberries
  - feta cheese
  - tarragon
  - (3 oz/90 g) finely chopped button or cremini mushrooms
  - rosemary
  - (2½ oz/75 g) cooked brown rice
  - chervil
  - (1½ oz/45 g) fresh or frozen corn kernels
  - (3½ oz/105 g) canned low-sodium black beans
  - (4 fl oz/125 ml) water
  - attieké
  - (2 oz/60 g) quinoa
  - (8 fl oz/250 ml) water
  - (oz/20 g) old-fashioned rolled oats
  - wild salmon fillet (about 4 oz/125 g)
  - (14½ oz/455 g) diced tomatoes
  - (10 oz/315 g) finely chopped butternut squash (about ½-inch/12-mm pieces)
  - (15½ oz/485 g) chickpeas
  - (8 fl oz/250 ml) low-sodium vegetable or chicken broth or water
  - (1½ oz/45 g) finely chopped kiwi
  - (1½ oz/45 g) finely chopped mango
  - (1½ oz/45 g) finely chopped
  - (2 oz/60 g) shredded mild white cheddar cheese
  - teaspoons of cornflour
  - (4 fl oz/125 ml) whole milk
  - agar-agar
  - (4 oz/125 g) broccoli florets
  - (8 fl oz/250 ml) unswee

In [316]:
# 📊 BATCH 6 PROCESSING (Ingredients 751+) - FINAL BATCH
print("🔄 PROCESSING BATCH 6 (FINAL)")
print("=" * 50)
print(f"Batch 6 contains {len(batch_dfs['batch_6'])} ingredients")
print(f"Sample ingredients: {list(batch_dfs['batch_6']['ingredient'].head())}")
print()

# Process Batch 6
batch_6_results, batch_6_failed = process_batch_dataframe(
    batch_dfs['batch_6'], 
    'batch_6', 
    api_key, 
    save_results=True
)

print(f"\n✅ BATCH 6 COMPLETED!")


🔄 PROCESSING BATCH 6 (FINAL)
Batch 6 contains 120 ingredients
Sample ingredients: ['new potatoes (1 portion)', 'white bread', 'turmeric', 'raisins (optional)', 'apricot jam']


🔄 PROCESSING BATCH_6
Processing 1/120: new potatoes (1 portion)
🔍 Step 1: Searching Foundation for exact match: 'new potatoes (1 portion)'
🔍 Step 2: Taking first Foundation result: 'new potatoes (1 portion)'
📄 Using first Foundation result: Flour, potato
  ✅ Success - Method: Foundation_first
Processing 2/120: white bread
🔍 Step 1: Searching Foundation for exact match: 'white bread'
🔍 Step 2: Taking first Foundation result: 'white bread'
📄 Using first Foundation result: Bread, white, commercially prepared
  ✅ Success - Method: Foundation_first
Processing 3/120: turmeric
🔍 Step 1: Searching Foundation for exact match: 'turmeric'
🔍 Step 2: Taking first Foundation result: 'turmeric'
🔍 Step 3: Searching Survey (FNDDS) for exact match: 'turmeric'
❌ No nutrition data found for 'turmeric'
  ❌ Failed
Processing 4/120: r

In [317]:
# 🔍 CHECK BATCH 5 RESULTS
print("📋 BATCH 6 DETAILED RESULTS")
print("=" * 50)

if 'batch_6_results' in locals():
    print(f"✅ Successfully processed: {len(batch_6_results[batch_6_results['search_method'].notna()])} ingredients")
    print(f"❌ Failed: {len(batch_5_failed)} ingredients")
    print(f"📊 Success rate: {len(batch_6_results[batch_6_results['search_method'].notna()]) / len(batch_6_results) * 100:.1f}%")
    
    # Show full results DataFrame
    print(f"\n📊 FULL BATCH 5 RESULTS:")
    display(batch_6_results)
    
    # Show failed ingredients if any
    if batch_6_failed:
        print(f"\n❌ FAILED INGREDIENTS IN BATCH 5:")
        for ingredient in batch_6_failed:
            print(f"  - {ingredient}")
    
    print(f"\n✅ Ready to proceed to Batch 6 (FINAL)? Run the next cell!")
else:
    print("⚠️ Batch 5 not processed yet. Run the previous cell first.")

📋 BATCH 6 DETAILED RESULTS
✅ Successfully processed: 89 ingredients
❌ Failed: 42 ingredients
📊 Success rate: 74.2%

📊 FULL BATCH 5 RESULTS:


,ingredient,description,energy_kcal_per_100g,carbs_g_per_100g,protein_g_per_100g,fat_g_per_100g,top_micronutrients,search_method,batch_number
0,new potatoes (1 portion),"Flour, potato",353,79.9,8.11,0.951,"Potassium, K, Phosphorus, P, Magnesium, Mg",Foundation_first,6
1,white bread,"Bread, white, commercially prepared",None,49.2,9.43,3.59,"Sodium, Na, Calcium, Ca, Potassium, K",Foundation_first,6
2,turmeric,None,None,None,None,None,None,None,6
3,raisins (optional),"Cookies, oatmeal, soft, with raisins",None,69.6,5.79,14.3,"Sodium, Na, Potassium, K, Phosphorus, P",Foundation_first,6
4,apricot jam,"Apricot, with skin, raw",43.5,10.2,0.961,0.405,"Potassium, K, Phosphorus, P, Calcium, Ca",Foundation_first,6
...,...,...,...,...,...,...,...,...,...
115,Natto,Natto,None,12.68,19.4,11,"Potassium, K, Calcium, Ca, Phosphorus, P",Survey_exact,6
116,Japanese rice *steamed,"Flour, rice, brown",None,75.5,7.19,3.85,"Phosphorus, P, Potassium, K, Magnesium, Mg",Foundation_first,6
117,dashi (baby-safe,None,None,None,None,None,None,None,6
118,udon (preferably thin,None,None,None,None,None,None,None,6



❌ FAILED INGREDIENTS IN BATCH 5:
  - turmeric
  - (0.35 oz) fresh salmon fillet
  - heavy cream (or crème fraîche)
  - slice wholemeal bread (30g)
  - 1/2 cups (28 fl oz/875 ml) beef or chicken broth
  - wholemeal self-raising flour (if you don’t have wholemeal use self-raising )
  - full-fat cream cheese (1 portion)
  - florets (85g)
  - broccoli (stalks removed
  - cream cheese
  - butterbeans
  - courgette (approx 200g)
  - cloves
  - (8 oz/250 g) sweet potato
  - 30% fat liquid cream
  - (50g)
  - plain full-fat yoghurt (one portion)
  - plain full fat yoghurt (optional)
  - full-fat yoghurt (optional) – put this in a bowl for your baby to use as a dip
  - sweetcorn (160g)
  - bisk
  - canned peach slices (in juice
  - baking potato (1/3 to serve)
  - tinned chopped tomatoes (1/4 can)
  - vegetables (bok choy
  - full-fat milk or baby's usual milk
  - (1 oz/30 g) baby oatmeal
  - dashi
  - Salmon
  - dashi (baby-safe
  - udon (preferably thin

✅ Ready to proceed to Batch 6 (FINAL)

## Combine All Individual Batch Results

This section contains functions to combine all individually processed batch results into a single Excel file with separate sheets for successful and failed results.

In [318]:
def combine_batch_results_to_single_file(output_filename=None):
    """
    Combine all individually processed batch results into a single Excel file 
    with 2 sheets: Successful and Failed results, each with batch tracking.
    
    This function looks for variables in the current namespace following the pattern:
    - batch_X_results (DataFrame with successful results)
    - batch_X_failed (List with failed ingredients)
    
    Parameters:
    - output_filename: Output filename (if None, auto-generated with timestamp)
    
    Returns:
    - combined_successful_df: DataFrame with all successful results
    - combined_failed_df: DataFrame with all failed results
    - summary_stats: Dictionary with combination statistics
    """
    import pandas as pd
    from datetime import datetime
    import re
    
    print("🔍 Searching for existing batch results...")
    
    # Get all variables from the current namespace
    current_vars = globals()
    
    # Find all batch result variables
    successful_batches = {}
    failed_batches = {}
    
    # Pattern to match batch variables
    for var_name in current_vars:
        # Match successful batch results (e.g., batch_1_results, batch_2_results)
        if re.match(r'batch_\d+_results$', var_name):
            batch_num = re.search(r'batch_(\d+)_results', var_name).group(1)
            batch_name = f"batch_{batch_num}"
            if isinstance(current_vars[var_name], pd.DataFrame):
                successful_batches[batch_name] = current_vars[var_name]
                print(f"   ✅ Found {var_name}: {len(current_vars[var_name])} successful results")
        
        # Match failed batch results (e.g., batch_1_failed, batch_2_failed)
        elif re.match(r'batch_\d+_failed$', var_name):
            batch_num = re.search(r'batch_(\d+)_failed', var_name).group(1)
            batch_name = f"batch_{batch_num}"
            if isinstance(current_vars[var_name], list):
                failed_batches[batch_name] = current_vars[var_name]
                print(f"   ❌ Found {var_name}: {len(current_vars[var_name])} failed ingredients")
    
    if not successful_batches and not failed_batches:
        print("⚠️  No batch results found! Make sure you have processed batches first.")
        print("   Expected variables: batch_1_results, batch_1_failed, batch_2_results, batch_2_failed, etc.")
        return None, None, None
    
    print(f"\n📊 Found {len(successful_batches)} successful batch result sets")
    print(f"📊 Found {len(failed_batches)} failed batch result sets")
    
    # Combine all successful results
    all_successful_dfs = []
    total_successful = 0
    
    for batch_name, df in successful_batches.items():
        if not df.empty:
            # Add batch column
            df_copy = df.copy()
            df_copy['batch'] = batch_name
            all_successful_dfs.append(df_copy)
            total_successful += len(df_copy)
            print(f"   ✅ {batch_name}: {len(df_copy)} successful results")
    
    # Combine all failed results
    all_failed_data = []
    total_failed = 0
    
    for batch_name, failed_list in failed_batches.items():
        if failed_list:
            for ingredient in failed_list:
                all_failed_data.append({
                    'ingredient': ingredient,
                    'batch': batch_name,
                    'reason': 'No nutrition data found',
                    'processed_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
            total_failed += len(failed_list)
            print(f"   ❌ {batch_name}: {len(failed_list)} failed ingredients")
    
    # Create combined DataFrames
    if all_successful_dfs:
        combined_successful_df = pd.concat(all_successful_dfs, ignore_index=True)
        # Reorder columns to put batch column first after ingredient
        cols = list(combined_successful_df.columns)
        if 'batch' in cols and 'ingredient' in cols:
            # Move batch column to second position (after ingredient)
            cols.remove('batch')
            ingredient_idx = cols.index('ingredient')
            cols.insert(ingredient_idx + 1, 'batch')
            combined_successful_df = combined_successful_df[cols]
    else:
        combined_successful_df = pd.DataFrame()
    
    if all_failed_data:
        combined_failed_df = pd.DataFrame(all_failed_data)
    else:
        combined_failed_df = pd.DataFrame()
    
    # Generate filename if not provided
    if output_filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_filename = f'combined_batch_results_{timestamp}.xlsx'
    
    # Save to Excel with multiple sheets
    print(f"\n💾 Saving combined results to: {output_filename}")
    
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        # Save successful results
        if not combined_successful_df.empty:
            combined_successful_df.to_excel(writer, sheet_name='Successful', index=False)
            print(f"   ✅ Successful sheet: {len(combined_successful_df)} ingredients saved")
        else:
            # Create empty sheet with headers
            pd.DataFrame(columns=['ingredient', 'batch', 'energy_kcal', 'carbs_g', 'protein_g', 'fat_g', 
                                'micronutrients', 'search_method', 'found_description']).to_excel(
                writer, sheet_name='Successful', index=False)
            print("   ⚠️  No successful results to save")
        
        # Save failed results
        if not combined_failed_df.empty:
            combined_failed_df.to_excel(writer, sheet_name='Failed', index=False)
            print(f"   ❌ Failed sheet: {len(combined_failed_df)} ingredients saved")
        else:
            # Create empty sheet with headers
            pd.DataFrame(columns=['ingredient', 'batch', 'reason', 'processed_at']).to_excel(
                writer, sheet_name='Failed', index=False)
            print("   ✅ No failed results (perfect success!)")
        
        # Create summary sheet
        summary_data = []
        batch_stats = {}
        
        # Get stats for each batch
        all_batch_names = set(list(successful_batches.keys()) + list(failed_batches.keys()))
        
        for batch_name in sorted(all_batch_names):
            successful_count = len(successful_batches.get(batch_name, pd.DataFrame()))
            failed_count = len(failed_batches.get(batch_name, []))
            total_count = successful_count + failed_count
            success_rate = (successful_count / total_count * 100) if total_count > 0 else 0
            
            summary_data.append({
                'Batch': batch_name,
                'Total_Ingredients': total_count,
                'Successful': successful_count,
                'Failed': failed_count,
                'Success_Rate_%': round(success_rate, 2)
            })
            
            batch_stats[batch_name] = {
                'total': total_count,
                'successful': successful_count,
                'failed': failed_count,
                'success_rate': success_rate
            }
        
        # Add overall summary
        overall_total = total_successful + total_failed
        if overall_total > 0:
            summary_data.append({
                'Batch': 'OVERALL_TOTAL',
                'Total_Ingredients': overall_total,
                'Successful': total_successful,
                'Failed': total_failed,
                'Success_Rate_%': round((total_successful / overall_total) * 100, 2)
            })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        print(f"   📊 Summary sheet: Batch statistics saved")
    
    # Print final summary
    print(f"\n🎉 COMBINATION COMPLETED!")
    print(f"📊 Final Combined Results:")
    print(f"   • Total ingredients: {total_successful + total_failed}")
    print(f"   • Successful: {total_successful} ({(total_successful/(total_successful + total_failed))*100:.1f}%)")
    print(f"   • Failed: {total_failed} ({(total_failed/(total_successful + total_failed))*100:.1f}%)")
    print(f"   • Batches combined: {len(all_batch_names)}")
    print(f"💾 Results saved to: {output_filename}")
    
    return combined_successful_df, combined_failed_df, {
        'total_ingredients': total_successful + total_failed,
        'successful': total_successful,
        'failed': total_failed,
        'success_rate': (total_successful / (total_successful + total_failed)) * 100 if (total_successful + total_failed) > 0 else 0,
        'batches_combined': len(all_batch_names),
        'batch_statistics': batch_stats,
        'output_filename': output_filename
    }

In [319]:
combined_successful_df, combined_failed_df, summary_stats = combine_batch_results_to_single_file(
    # output_filename=None  # Auto-generate filename with timestamp
    output_filename='my_combined_results.xlsx'  # Or specify custom filename
)

if combined_successful_df is not None:
    # Display final summary
    print(f"\n📊 FINAL COMBINATION SUMMARY:")
    print(f"✅ Total successful: {len(combined_successful_df)}")
    print(f"❌ Total failed: {len(combined_failed_df)}")
    print(f"📈 Overall success rate: {summary_stats['success_rate']:.1f}%")
    print(f"🔢 Batches combined: {summary_stats['batches_combined']}")
    print(f"💾 Results saved to: {summary_stats['output_filename']}")
    
    # Show batch breakdown
    print(f"\n📋 BATCH BREAKDOWN:")
    for batch_name, stats in summary_stats['batch_statistics'].items():
        print(f"   {batch_name}: {stats['successful']}/{stats['total']} successful ({stats['success_rate']:.1f}%)")
    
    # Display sample successful results
    if not combined_successful_df.empty:
        print(f"\n📄 SAMPLE SUCCESSFUL RESULTS (first 5):")
        display(combined_successful_df.head())
    
    # Display sample failed results  
    if not combined_failed_df.empty:
        print(f"\n❌ SAMPLE FAILED RESULTS (first 5):")
        display(combined_failed_df.head())
else:
    print("❌ No batch results found to combine. Please process some batches first!")

🔍 Searching for existing batch results...
   ✅ Found batch_1_results: 150 successful results
   ❌ Found batch_1_failed: 23 failed ingredients
   ✅ Found batch_2_results: 150 successful results
   ❌ Found batch_2_failed: 32 failed ingredients
   ✅ Found batch_3_results: 150 successful results
   ❌ Found batch_3_failed: 27 failed ingredients
   ✅ Found batch_4_results: 150 successful results
   ❌ Found batch_4_failed: 34 failed ingredients
   ✅ Found batch_5_results: 150 successful results
   ❌ Found batch_5_failed: 42 failed ingredients
   ✅ Found batch_6_results: 120 successful results
   ❌ Found batch_6_failed: 31 failed ingredients

📊 Found 6 successful batch result sets
📊 Found 6 failed batch result sets
   ✅ batch_1: 150 successful results
   ✅ batch_2: 150 successful results
   ✅ batch_3: 150 successful results
   ✅ batch_4: 150 successful results
   ✅ batch_5: 150 successful results
   ✅ batch_6: 120 successful results
   ❌ batch_1: 23 failed ingredients
   ❌ batch_2: 32 failed i

PermissionError: [Errno 13] Permission denied: 'my_combined_results.xlsx'

## Request to do Specific Ingredients

doing crosschecking each ingredients that are missed/not make sense

In [ ]:
BASE_URL = 'https://api.nal.usda.gov/fdc/v1/food/'

def get_food_data(fdc_id):
    """Fetch food data from USDA FoodData Central using the given FDC ID."""
    url = f"{BASE_URL}{fdc_id}"
    params = {
        'api_key': api_key,
        'format': 'abridged',  # This ensures we get nutrient values
        'nutrients': 'all'     # Get all nutrients with values
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [ ]:
def extract_nutrition_info(food, search_method):
    """Extract nutrition information from food item."""
    nutrients = food.get('foodNutrients', [])
    
    result = {
        'ingredient_name': food.get('description', 'N/A'),
        'found_description': food.get('description', 'N/A'),
        'search_method': food.get('foodClass', search_method),  # Use foodClass as search_method
        'energy_kcal': None,
        'carbohydrate_g': None,
        'protein_g': None,
        'fat_g': None,
        'micronutrients': [],
        'status': 'success'
    }
    
    # Energy (Atwater General Factors)
    energy = next((item for item in nutrients if item['nutrient']['name'] == 'Energy (Atwater Specific Factors)'), None)
    if energy:
        result['energy_kcal'] = energy['amount']
    
    # Carbohydrates
    carbohydrate = next((item for item in nutrients if item['nutrient']['name'] == 'Carbohydrate, by difference'), None)
    if carbohydrate:
        result['carbohydrate_g'] = carbohydrate['amount']
    
    # Fat
    fat = next((item for item in nutrients if item['nutrient']['name'] == 'Total lipid (fat)'), None)
    if fat:
        result['fat_g'] = fat['amount']
    
    # Protein
    protein = next((item for item in nutrients if item['nutrient']['name'] == 'Protein'), None)
    if protein:
        result['protein_g'] = protein['amount']
    
    # Exclude certain nutrients from micronutrients
    exclude_nutrients = [
        "Energy", "Water", "Energy (Atwater General Factors)", "Energy (Atwater Specific Factors)",
        "Nitrogen", "Protein", "Total lipid (fat)", "Ash", "Carbohydrates",
        "Carbohydrate, by difference", "Total dietary fiber (AOAC 2011.25)",
        "High Molecular Weight Dietary Fiber (HMWDF)", "Low Molecular Weight Dietary Fiber (LMWDF)",
        "Sugars, Total", "Total Sugars", "Sucrose", "Glucose", "Fructose", "Lactose", "Maltose"
    ]
    
    # Get micronutrients (vitamins and minerals) - top 3 by amount
    filtered_micronutrients = [
        item for item in nutrients 
        if item['nutrient']['name'] not in exclude_nutrients and item['amount'] > 0
    ]
    
    # Sort by amount in descending order and take top 3
    sorted_micronutrients = sorted(filtered_micronutrients, key=lambda x: x['amount'], reverse=True)
    top_3_micronutrients = sorted_micronutrients[:3]
    
    # Extract only the nutrient names
    micronutrients = [item['nutrient']['name'] for item in top_3_micronutrients]
    result['micronutrients'] = micronutrients
    
    return result

In [ ]:
# def get_food_nutrition(fdc_id, search_method):
#     food_data = get_food_data(fdc_id)
#     if food_data:
#         return extract_nutrition_info(food_data, search_method)
#     else:
#         return {'status': 'failure', 'message': 'Failed to retrieve data'}

# # Example usage
# fdc_id = 2684441  # The FDC ID for the food item you're interested in
# search_method = 'direct_search'  # This could be any search method you'd like to track

# nutrition_info = get_food_nutrition(fdc_id, search_method)
# print(nutrition_info)

KeyError: 'amount'

In [ ]:
def extract_nutrition_info_robust(food, search_method):
    """Extract nutrition information from food item with robust structure handling."""
    nutrients = food.get('foodNutrients', [])
    
    result = {
        'ingredient_name': food.get('description', 'N/A'),
        'found_description': food.get('description', 'N/A'),
        'search_method': food.get('foodClass', search_method),
        'energy_kcal': None,
        'carbohydrate_g': None,
        'protein_g': None,
        'fat_g': None,
        'micronutrients': [],
        'status': 'success'
    }
    
    def get_nutrient_value(nutrients, nutrient_name):
        """Helper function to extract nutrient value regardless of structure."""
        for item in nutrients:
            # Handle different possible structures
            name = None
            value = None
            
            # Try different ways to get nutrient name
            if isinstance(item, dict):
                if 'nutrient' in item and isinstance(item['nutrient'], dict):
                    name = item['nutrient'].get('name')
                elif 'nutrientName' in item:
                    name = item['nutrientName']
                elif 'name' in item:
                    name = item['name']
                
                # Try different ways to get value
                if 'amount' in item:
                    value = item['amount']
                elif 'value' in item:
                    value = item['value']
                elif 'quantity' in item:
                    value = item['quantity']
            
            # Check if this is the nutrient we're looking for
            if name and nutrient_name.lower() in name.lower():
                return value
        return None
    
    # Extract main nutrients
    result['energy_kcal'] = get_nutrient_value(nutrients, 'Energy (Atwater General Factors)')
    if result['energy_kcal'] is None:
        result['energy_kcal'] = get_nutrient_value(nutrients, 'Energy')
    
    result['carbohydrate_g'] = get_nutrient_value(nutrients, 'Carbohydrate, by difference')
    if result['carbohydrate_g'] is None:
        result['carbohydrate_g'] = get_nutrient_value(nutrients, 'Carbohydrate')
    
    result['fat_g'] = get_nutrient_value(nutrients, 'Total lipid (fat)')
    if result['fat_g'] is None:
        result['fat_g'] = get_nutrient_value(nutrients, 'Fat')
    
    result['protein_g'] = get_nutrient_value(nutrients, 'Protein')
    
    # Get micronutrients (excluding main macronutrients)
    exclude_nutrients = [
        "Energy", "Water", "Energy (Atwater General Factors)", "Energy (Atwater Specific Factors)",
        "Nitrogen", "Protein", "Total lipid (fat)", "Fat", "Ash", "Carbohydrates",
        "Carbohydrate, by difference", "Total dietary fiber", "Fiber",
        "Sugars", "Sugar", "Sucrose", "Glucose", "Fructose", "Lactose", "Maltose"
    ]
    
    micronutrients_with_values = []
    
    for item in nutrients:
        if isinstance(item, dict):
            name = None
            value = None
            
            # Get nutrient name
            if 'nutrient' in item and isinstance(item['nutrient'], dict):
                name = item['nutrient'].get('name')
            elif 'nutrientName' in item:
                name = item['nutrientName']
            elif 'name' in item:
                name = item['name']
            
            # Get value
            if 'amount' in item:
                value = item['amount']
            elif 'value' in item:
                value = item['value']
            elif 'quantity' in item:
                value = item['quantity']
            
            # Check if this is a micronutrient
            if name and value is not None and value > 0:
                is_excluded = any(excl.lower() in name.lower() for excl in exclude_nutrients)
                if not is_excluded:
                    micronutrients_with_values.append({'name': name, 'value': value})
    
    # Sort by value and take top 3
    sorted_micronutrients = sorted(micronutrients_with_values, key=lambda x: x['value'], reverse=True)
    top_3_micronutrients = sorted_micronutrients[:3]
    result['micronutrients'] = [item['name'] for item in top_3_micronutrients]
    
    return result

# Test the robust function
print("🧪 Testing robust extraction function:")
fdc_id = 2684441
food_data = get_food_data(fdc_id)
if food_data:
    nutrition_info = extract_nutrition_info_robust(food_data, 'direct_search')
    print("✅ Success! Nutrition info extracted:")
    for key, value in nutrition_info.items():
        print(f"  {key}: {value}")
else:
    print("❌ Failed to get food data")

🧪 Testing robust extraction function:
✅ Success! Nutrition info extracted:
  ingredient_name: Fish, salmon, Atlantic, farm raised, raw
  found_description: Fish, salmon, Atlantic, farm raised, raw
  search_method: FinalFood
  energy_kcal: 197.294
  carbohydrate_g: 0
  protein_g: 20.31875
  fat_g: 13.11
  micronutrients: ['Potassium, K', 'Phosphorus, P', 'Cholesterol']
  status: success


In [ ]:
# Let's examine several nutrients to understand the structure
food_data = get_food_data(2684441)
if food_data:
    nutrients = food_data.get('foodNutrients', [])
    print(f"🔍 DETAILED NUTRIENT INSPECTION (first 10 nutrients):")
    
    for i, nutrient in enumerate(nutrients[:10]):
        print(f"\n--- Nutrient {i+1} ---")
        print(f"Full structure: {nutrient}")
        
        if 'nutrient' in nutrient:
            print(f"Nutrient name: {nutrient['nutrient'].get('name')}")
        
        # Check for any field containing values
        value_fields = [key for key in nutrient.keys() if 'value' in key.lower() or 'amount' in key.lower() or 'quantity' in key.lower()]
        if value_fields:
            print(f"Value fields found: {value_fields}")
            for field in value_fields:
                print(f"  {field}: {nutrient[field]}")
        else:
            print("No value fields found")
            
    # Let's also check if there's a different way to get nutrient values
    print(f"\n🔍 Looking for nutrients with actual values...")
    nutrients_with_values = []
    for nutrient in nutrients:
        if any(key for key in nutrient.keys() if 'value' in key.lower() or 'amount' in key.lower()):
            nutrients_with_values.append(nutrient)
    
    print(f"Found {len(nutrients_with_values)} nutrients with value fields")
    if nutrients_with_values:
        print(f"Example nutrient with values: {nutrients_with_values[0]}")

🔍 DETAILED NUTRIENT INSPECTION (first 10 nutrients):

--- Nutrient 1 ---
Full structure: {'nutrient': {'id': 2045, 'number': '951', 'name': 'Proximates', 'rank': 50, 'unitName': 'g'}, 'type': 'FoodNutrient'}
Nutrient name: Proximates
No value fields found

--- Nutrient 2 ---
Full structure: {'type': 'FoodNutrient', 'nutrient': {'id': 1051, 'number': '255', 'name': 'Water', 'rank': 100, 'unitName': 'g'}, 'foodNutrientDerivation': {'id': 1, 'code': 'A', 'description': 'Analytical', 'foodNutrientSource': {'id': 1, 'code': '1', 'description': 'Analytical or derived from analytical'}}, 'id': 33829175, 'amount': 65.84, 'dataPoints': 8, 'max': 69.14, 'min': 62.97, 'median': 65.95, 'minYearAcquired': 2023, 'nutrientAnalysisDetails': [{'subSampleId': 2684576, 'nutrientId': 1051, 'nutrientAcquisitionDetails': [{'sampleUnitId': 2684479, 'purchaseDate': '9/6/2023', 'storeCity': 'Blacksburg', 'storeState': 'VA', 'packerCity': 'Cincinnati', 'packerState': 'OH'}], 'amount': 62.97, 'labMethodTechnique

In [ ]:
# Test with updated API call
print("🧪 Testing updated API call...")
fdc_id = 2684441
food_data = get_food_data(fdc_id)

if food_data:
    nutrients = food_data.get('foodNutrients', [])
    print(f"Number of nutrients: {len(nutrients)}")
    
    # Look for nutrients with actual values
    print(f"\n🔍 First few nutrients with their full structure:")
    for i, nutrient in enumerate(nutrients[:3]):
        print(f"\nNutrient {i+1}: {nutrient}")
        
        # Check all possible fields
        print(f"All keys: {list(nutrient.keys())}")
        
        # Look for value in any field
        for key, value in nutrient.items():
            if isinstance(value, (int, float)) and value != 0:
                print(f"  Possible value field '{key}': {value}")
else:
    print("❌ Failed to get food data")

# Try a direct API call to see raw response
print(f"\n🌐 Direct API call test:")
import requests
url = f"https://api.nal.usda.gov/fdc/v1/food/{fdc_id}"
params = {'api_key': api_key}
response = requests.get(url, params=params)
print(f"Status: {response.status_code}")
if response.status_code == 200:
    data = response.json()
    nutrients = data.get('foodNutrients', [])
    if nutrients:
        print(f"First nutrient from direct call: {nutrients[0]}")
    else:
        print("No nutrients in direct call")

🧪 Testing updated API call...
Number of nutrients: 45

🔍 First few nutrients with their full structure:

Nutrient 1: {'nutrient': {'id': 2045, 'number': '951', 'name': 'Proximates', 'rank': 50, 'unitName': 'g'}, 'type': 'FoodNutrient'}
All keys: ['nutrient', 'type']

Nutrient 2: {'type': 'FoodNutrient', 'nutrient': {'id': 1051, 'number': '255', 'name': 'Water', 'rank': 100, 'unitName': 'g'}, 'foodNutrientDerivation': {'id': 1, 'code': 'A', 'description': 'Analytical', 'foodNutrientSource': {'id': 1, 'code': '1', 'description': 'Analytical or derived from analytical'}}, 'id': 33829175, 'amount': 65.84, 'dataPoints': 8, 'max': 69.14, 'min': 62.97, 'median': 65.95, 'minYearAcquired': 2023, 'nutrientAnalysisDetails': [{'subSampleId': 2684576, 'nutrientId': 1051, 'nutrientAcquisitionDetails': [{'sampleUnitId': 2684479, 'purchaseDate': '9/6/2023', 'storeCity': 'Blacksburg', 'storeState': 'VA', 'packerCity': 'Cincinnati', 'packerState': 'OH'}], 'amount': 62.97, 'labMethodTechnique': 'Vacuum o

## Batch Process Specific FDC IDs

Function to process a list of [name, id] pairs, fetch nutrition data for each ID, and save to Excel.

In [ ]:
# CORRECTED VERSION - Replace the original extract_nutrition_info function with this
def extract_nutrition_info_corrected(food, search_method):
    """Extract nutrition information from food item - CORRECTED VERSION."""
    nutrients = food.get('foodNutrients', [])
    
    result = {
        'ingredient_name': food.get('description', 'N/A'),
        'found_description': food.get('description', 'N/A'),
        'search_method': food.get('foodClass', search_method),
        'energy_kcal': None,
        'carbohydrate_g': None,
        'protein_g': None,
        'fat_g': None,
        'micronutrients': [],
        'status': 'success'
    }
    
    def get_nutrient_value(nutrient_item):
        """Get the value from a nutrient item, handling different field names."""
        # Try different possible field names for the value
        possible_fields = ['amount', 'value', 'quantity', 'val']
        for field in possible_fields:
            if field in nutrient_item:
                return nutrient_item[field]
        return None
    
    # 1st priority: Energy (Atwater Specific Factors)
    energy = next((item for item in nutrients if item['nutrient']['name'] == 'Energy (Atwater Specific Factors)'), None)
    if energy:
        result['energy_kcal'] = get_nutrient_value(energy)
        result['energy_source'] = 'Atwater Specific Factors'
        print(f"   ✅ Found Energy (Atwater Specific Factors): {result['energy_kcal']} kcal")
    else:
        # 2nd priority: Energy (Atwater General Factors)
        energy = next((item for item in nutrients if item['nutrient']['name'] == 'Energy (Atwater General Factors)'), None)
        if energy:
            result['energy_kcal'] = get_nutrient_value(energy)
            result['energy_source'] = 'Atwater General Factors'
            print(f"   ✅ Found Energy (Atwater General Factors): {result['energy_kcal']} kcal")
        else:
            # 3rd priority: Energy with unitName = "kcal" ⭐ YOUR REQUIREMENT
            energy = next((item for item in nutrients 
                          if item['nutrient']['name'] == 'Energy' and 
                          item['nutrient'].get('unitName') == 'kcal'), None)
            if energy:
                result['energy_kcal'] = get_nutrient_value(energy)
                result['energy_source'] = 'Energy (kcal unit)'
                print(f"   ✅ Found Energy with unitName='kcal': {result['energy_kcal']} kcal")
            else:
                # 4th priority: Any Energy entry as final fallback
                energy = next((item for item in nutrients if 'Energy' in item['nutrient']['name']), None)
                if energy:
                    result['energy_kcal'] = get_nutrient_value(energy)
                    unit = energy['nutrient'].get('unitName', 'unknown unit')
                    result['energy_source'] = f'Fallback Energy ({unit})'
                    print(f"   ⚠️ Found fallback Energy: {result['energy_kcal']} {unit}")
                else:
                    result['energy_source'] = 'Not found'
                    print(f"   ❌ No Energy data found")
    
    # Extract other macronutrients
    print(f"   🔍 Extracting other nutrients...")
    
    # Carbohydrates
    carbohydrate = next((item for item in nutrients if item['nutrient']['name'] == 'Carbohydrate, by difference'), None)
    if carbohydrate:
        result['carbohydrate_g'] = get_nutrient_value(carbohydrate)
    
    # Fat
    fat = next((item for item in nutrients if item['nutrient']['name'] == 'Total lipid (fat)'), None)
    if fat:
        result['fat_g'] = get_nutrient_value(fat)
    
    # Protein
    protein = next((item for item in nutrients if item['nutrient']['name'] == 'Protein'), None)
    if protein:
        result['protein_g'] = get_nutrient_value(protein)
    
    # Exclude certain nutrients from micronutrients
    exclude_nutrients = [
        "Energy", "Water", "Energy (Atwater General Factors)", "Energy (Atwater Specific Factors)",
        "Nitrogen", "Protein", "Total lipid (fat)", "Ash", "Carbohydrates",
        "Carbohydrate, by difference", "Total dietary fiber (AOAC 2011.25)",
        "High Molecular Weight Dietary Fiber (HMWDF)", "Low Molecular Weight Dietary Fiber (LMWDF)",
        "Sugars, Total", "Total Sugars", "Sucrose", "Glucose", "Fructose", "Lactose", "Maltose"
    ]
    
    # Get micronutrients (vitamins and minerals) - top 3 by value
    filtered_micronutrients = []
    for item in nutrients:
        if (item['nutrient']['name'] not in exclude_nutrients and 
            get_nutrient_value(item) is not None and 
            get_nutrient_value(item) > 0):
            filtered_micronutrients.append({
                'name': item['nutrient']['name'],
                'value': get_nutrient_value(item)
            })
    
    # Sort by value in descending order and take top 3
    sorted_micronutrients = sorted(filtered_micronutrients, key=lambda x: x['value'], reverse=True)
    top_3_micronutrients = sorted_micronutrients[:3]
    
    # Extract only the nutrient names
    micronutrients = [item['name'] for item in top_3_micronutrients]
    result['micronutrients'] = micronutrients
    
    return result

# Test the corrected function
print("🧪 Testing CORRECTED extraction function:")
fdc_id = 2684441
food_data = get_food_data(fdc_id)
if food_data:
    nutrition_info = extract_nutrition_info_corrected(food_data, 'direct_search')
    print("✅ Success! Nutrition info extracted:")
    for key, value in nutrition_info.items():
        print(f"  {key}: {value}")
else:
    print("❌ Failed to get food data")

🧪 Testing CORRECTED extraction function:
   ✅ Found Energy (Atwater Specific Factors): 203.10632 kcal
   🔍 Extracting other nutrients...
✅ Success! Nutrition info extracted:
  ingredient_name: Fish, salmon, Atlantic, farm raised, raw
  found_description: Fish, salmon, Atlantic, farm raised, raw
  search_method: FinalFood
  energy_kcal: 203.10632
  carbohydrate_g: 0
  protein_g: 20.31875
  fat_g: 13.11
  micronutrients: ['Potassium, K', 'Phosphorus, P', 'Cholesterol']
  status: success
  energy_source: Atwater Specific Factors
   ✅ Found Energy (Atwater Specific Factors): 203.10632 kcal
   🔍 Extracting other nutrients...
✅ Success! Nutrition info extracted:
  ingredient_name: Fish, salmon, Atlantic, farm raised, raw
  found_description: Fish, salmon, Atlantic, farm raised, raw
  search_method: FinalFood
  energy_kcal: 203.10632
  carbohydrate_g: 0
  protein_g: 20.31875
  fat_g: 13.11
  micronutrients: ['Potassium, K', 'Phosphorus, P', 'Cholesterol']
  status: success
  energy_source: At

In [ ]:
def process_fdc_id_list_to_excel(name_id_list, output_filename=None, delay=0.5):
    """
    Process a list of [name, id] pairs, fetch nutrition data for each FDC ID, and save to Excel.
    
    Parameters:
    - name_id_list: List of [name, fdc_id] pairs, e.g., [["Apple", 123456], ["Banana", 789012]]
    - output_filename: Output Excel filename (if None, auto-generated with timestamp)
    - delay: Delay between API calls in seconds (default 0.5s)
    
    Returns:
    - results_df: DataFrame with all results
    - failed_list: List of failed items
    - summary_stats: Dictionary with processing statistics
    """
    import pandas as pd
    import time
    from datetime import datetime
    
    print(f"🚀 Processing {len(name_id_list)} FDC ID entries...")
    print("=" * 60)
    
    results = []
    failed_items = []
    
    start_time = time.time()
    
    for i, (name, fdc_id) in enumerate(name_id_list, 1):
        print(f"\n🔄 Processing {i}/{len(name_id_list)}: {name} (ID: {fdc_id})")
        
        try:
            # Convert fdc_id to int if it's a string
            if isinstance(fdc_id, str):
                fdc_id = int(fdc_id)
            
            # Get food data from API
            food_data = get_food_data(fdc_id)
            
            if food_data:
                # Extract nutrition info using the corrected function
                nutrition_info = extract_nutrition_info_corrected(food_data, 'FDC_ID_lookup')
                
                if nutrition_info and nutrition_info.get('status') == 'success':
                    # Create result entry
                    result_entry = {
                        'name': name,
                        'fdc_id': fdc_id,
                        'found_description': nutrition_info.get('found_description', ''),
                        'search_method': nutrition_info.get('search_method', ''),
                        'energy_kcal': nutrition_info.get('energy_kcal'),
                        'carbohydrate_g': nutrition_info.get('carbohydrate_g'),
                        'protein_g': nutrition_info.get('protein_g'),
                        'fat_g': nutrition_info.get('fat_g'),
                        'micronutrients': ', '.join(nutrition_info.get('micronutrients', [])),
                        'processed_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'status': 'success'
                    }
                    results.append(result_entry)
                    print(f"   ✅ Success: {nutrition_info.get('energy_kcal')} kcal, {len(nutrition_info.get('micronutrients', []))} micronutrients")
                else:
                    # Failed to extract nutrition
                    failed_entry = {
                        'name': name,
                        'fdc_id': fdc_id,
                        'reason': 'Failed to extract nutrition data',
                        'processed_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    }
                    failed_items.append(failed_entry)
                    print(f"   ❌ Failed: Could not extract nutrition data")
            else:
                # Failed to get food data
                failed_entry = {
                    'name': name,
                    'fdc_id': fdc_id,
                    'reason': 'Failed to fetch food data from API',
                    'processed_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }
                failed_items.append(failed_entry)
                print(f"   ❌ Failed: API call failed")
        
        except Exception as e:
            # Error occurred
            failed_entry = {
                'name': name,
                'fdc_id': fdc_id,
                'reason': f'Error: {str(e)}',
                'processed_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            failed_items.append(failed_entry)
            print(f"   ❌ Error: {str(e)}")
        
        # Add delay between requests
        if i < len(name_id_list):  # Don't delay after the last item
            time.sleep(delay)
    
    # Create DataFrames
    results_df = pd.DataFrame(results) if results else pd.DataFrame()
    failed_df = pd.DataFrame(failed_items) if failed_items else pd.DataFrame()
    
    # Generate filename if not provided
    if output_filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_filename = f'fdc_nutrition_data_{timestamp}.xlsx'
    
    # Save to Excel
    print(f"\n💾 Saving results to: {output_filename}")
    
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        # Save successful results
        if not results_df.empty:
            results_df.to_excel(writer, sheet_name='Nutrition_Data', index=False)
            print(f"   ✅ Nutrition data saved: {len(results_df)} items")
        else:
            # Create empty sheet with headers
            empty_df = pd.DataFrame(columns=['name', 'fdc_id', 'found_description', 'search_method', 
                                           'energy_kcal', 'carbohydrate_g', 'protein_g', 'fat_g', 
                                           'micronutrients', 'processed_at', 'status'])
            empty_df.to_excel(writer, sheet_name='Nutrition_Data', index=False)
            print("   ⚠️  No successful results to save")
        
        # Save failed items
        if not failed_df.empty:
            failed_df.to_excel(writer, sheet_name='Failed_Items', index=False)
            print(f"   ❌ Failed items saved: {len(failed_df)} items")
        
        # Create summary sheet
        summary_data = [{
            'Total_Items': len(name_id_list),
            'Successful': len(results),
            'Failed': len(failed_items),
            'Success_Rate_%': (len(results) / len(name_id_list)) * 100 if name_id_list else 0,
            'Processing_Time_Seconds': round(time.time() - start_time, 1),
            'Processed_At': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }]
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        print(f"   📊 Summary saved")
    
    # Print final summary
    total_time = time.time() - start_time
    success_rate = (len(results) / len(name_id_list)) * 100 if name_id_list else 0
    
    print(f"\n🎉 PROCESSING COMPLETED!")
    print(f"📊 Final Results:")
    print(f"   • Total items processed: {len(name_id_list)}")
    print(f"   • Successful: {len(results)} ({success_rate:.1f}%)")
    print(f"   • Failed: {len(failed_items)} ({100-success_rate:.1f}%)")
    print(f"   • Total processing time: {total_time:.1f} seconds")
    print(f"   • Average time per item: {total_time/len(name_id_list):.2f} seconds")
    print(f"💾 Results saved to: {output_filename}")
    
    return results_df, failed_items, {
        'total_items': len(name_id_list),
        'successful': len(results),
        'failed': len(failed_items),
        'success_rate': success_rate,
        'processing_time': total_time,
        'output_filename': output_filename
    }

In [ ]:
name_id_list = [
    ["salmon", 2684441],
    ["Whole Milk", 746782],
    ["White Bread", 339005],
    ["yoghurt", 2259793],
    ["baby oatmeal", 2708492],
    ["bok choy", 2685572],
    ["tomato", 2685581],
    ["clove", 171321],
    ["cloves", 171321],
    ["wholemeal bread", 335240],
    ["rice",2512381],
    ["basmati rice", 2708404],
    ["thyme", 173470], 
    ["coconut milk", 2705413],
    ["paprika", 171329],
    ["cod", 2684444],
    ["dark chocolat",170271],
    ["baking powder", 172805],
    ["formula", 2705518],
    ["parsley", 170416], 
    ["butter", 790508],
    ["apricot", 2710815],
    ["stock",2707132],
    ["coriander", 169997],
    ["chayote", 170402],
    ["chives", 169994],
    ["blueberry", 2346411],
    ["blueberries", 2346411],
    ["spring onion", 170005],
    ["shrimp", 2684443],
    ["catfish", 2684445],
    ["brown sugar", 2710260],
    ["berries", 2709272],
    ["turkey", 2514747],
    ["black pepper", 170931],
    ["raisins",2709212],
    ["egg noodles", 169731],
    ["lemon grass", 168573],
    ["papaya", 2709246],
    ["coconut water", 2707572],
    ["ginger", 169231],
    ["seaweed", 2709988],
    ["eyes fish", 168034],
    ["curry powder", 170924],
    ["cream cheese", 173418],
    ["feta cheese", 2259796],
    ["quinoa", 2708401],
    ["kiwi", 2710831],
    ["mango", 2710834],
    ["mangoes", 2710834],
    ["cassava", 169985], 
    
]
# Process the test list
results_df, failed_list, stats = process_fdc_id_list_to_excel(
    name_id_list=name_id_list,
    output_filename=None,  # Auto-generate filename
    delay=0.5  # 500ms delay between calls
)

🚀 Processing 51 FDC ID entries...

🔄 Processing 1/51: salmon (ID: 2684441)
   ✅ Found Energy (Atwater Specific Factors): 203.10632 kcal
   🔍 Extracting other nutrients...
   ✅ Success: 203.10632 kcal, 3 micronutrients
   ✅ Found Energy (Atwater Specific Factors): 203.10632 kcal
   🔍 Extracting other nutrients...
   ✅ Success: 203.10632 kcal, 3 micronutrients

🔄 Processing 2/51: Whole Milk (ID: 746782)

🔄 Processing 2/51: Whole Milk (ID: 746782)
   ✅ Found Energy with unitName='kcal': 60.0 kcal
   🔍 Extracting other nutrients...
   ✅ Success: 60.0 kcal, 3 micronutrients
   ✅ Found Energy with unitName='kcal': 60.0 kcal
   🔍 Extracting other nutrients...
   ✅ Success: 60.0 kcal, 3 micronutrients

🔄 Processing 3/51: White Bread (ID: 339005)

🔄 Processing 3/51: White Bread (ID: 339005)
   ❌ Failed: API call failed
   ❌ Failed: API call failed

🔄 Processing 4/51: yoghurt (ID: 2259793)

🔄 Processing 4/51: yoghurt (ID: 2259793)
   ✅ Found Energy (Atwater Specific Factors): 77.3185458 kcal
   

In [ ]:
# Display results
if not results_df.empty:
    print(f"\n📄 RESULTS PREVIEW:")
    display(results_df)
    
    print(f"\n📊 NUTRITION DATA SUMMARY:")
    numeric_cols = ['energy_kcal', 'carbohydrate_g', 'protein_g', 'fat_g']
    for col in numeric_cols:
        if col in results_df.columns:
            avg_val = results_df[col].mean()
            print(f"   • Average {col}: {avg_val:.2f}")

if failed_list:
    print(f"\n❌ FAILED ITEMS:")
    for item in failed_list:
        print(f"   • {item['name']} (ID: {item['fdc_id']}): {item['reason']}")

print(f"\n💾 Excel file saved: {stats['output_filename']}")
print(f"📈 Success rate: {stats['success_rate']:.1f}%")


📄 RESULTS PREVIEW:


,name,fdc_id,found_description,search_method,energy_kcal,carbohydrate_g,protein_g,fat_g,micronutrients,processed_at,status
0,salmon,2684441,"Fish, salmon, Atlantic, farm raised, raw",FinalFood,197.2940,0.000000,20.318750,13.1100,"Potassium, K, Phosphorus, P, Cholesterol",2025-06-23 14:59:33,success
1,Whole Milk,746782,"Milk, whole, 3.25% milkfat, with added vitamin D",FinalFood,NaN,4.630000,3.270000,3.2000,"Potassium, K, Calcium, Ca, Phosphorus, P",2025-06-23 15:00:04,success
2,yoghurt,2259793,"Yogurt, plain, whole milk",FinalFood,77.9524,5.574928,3.824172,4.4840,"Potassium, K, Calcium, Ca, Phosphorus, P",2025-06-23 15:00:09,success
3,baby oatmeal,2708492,"Baby Toddler cereal, oatmeal, dry",Survey,NaN,73.500000,11.000000,6.3600,"Calcium, Ca, Potassium, K, Phosphorus, P",2025-06-23 15:00:11,success
4,bok choy,2685572,"Cabbage, bok choy, raw",FinalFood,20.2590,3.514950,1.023750,0.2338,"Potassium, K, Folate, total, Calcium, Ca",2025-06-23 15:00:13,success
5,tomato,2685581,"Tomatoes, crushed, canned",FinalFood,37.8755,7.137000,1.437500,0.3975,"Potassium, K, Sodium, Na, Phosphorus, P",2025-06-23 15:00:16,success
6,clove,171321,"Spices, cloves, ground",FinalFood,NaN,65.530000,5.970000,13.0000,"Potassium, K, Calcium, Ca, Sodium, Na",2025-06-23 15:00:18,success
7,cloves,171321,"Spices, cloves, ground",FinalFood,NaN,65.530000,5.970000,13.0000,"Potassium, K, Calcium, Ca, Sodium, Na",2025-06-23 15:00:19,success
8,wholemeal bread,335240,"Bread, whole-wheat, commercially prepared",FinalFood,NaN,43.100000,12.300000,3.5500,"Sodium, Na, Potassium, K, Phosphorus, P",2025-06-23 15:00:27,success
9,rice,2512381,"Rice, white, long grain, unenriched, raw",FinalFood,358.7050,80.313150,7.038850,1.0330,"Phosphorus, P, Potassium, K, Starch",2025-06-23 15:00:30,success



📊 NUTRITION DATA SUMMARY:
   • Average energy_kcal: 111.39
   • Average carbohydrate_g: 25.21
   • Average protein_g: 6.48
   • Average fat_g: 6.32

❌ FAILED ITEMS:
   • White Bread (ID: 339005): Failed to fetch food data from API

💾 Excel file saved: fdc_nutrition_data_20250623_150143.xlsx
📈 Success rate: 98.0%


## Compute Nutrients Per Ingredient of The Recipe

In [ ]:
import pandas as pd

# Load the Excel file structure
excel_file = pd.ExcelFile('final_ingredient_nutrition.xlsx')

# Get list of available sheet names
sheet_names = excel_file.sheet_names

# Check if 'Succesful' sheet exists
if 'Successful' in sheet_names:
    combined_nutrition_data = pd.read_excel('final_ingredient_nutritions.xlsx', sheet_name='Successful')
    print("Shape:", combined_nutrition_data.shape)
    print("✅ Loaded Successful sheet.")
else:
    print("❌ Sheet 'Succesful' not found.")
    print("📄 Available sheets:", sheet_names)
    combined_nutrition_data = None  # or raise an error / fallback action


Shape: (712, 11)
✅ Loaded Successful sheet.


In [ ]:
# Filter for complete data
# Check ingredient breakdown data
print(f"\n=== INGREDIENT BREAKDOWN DATA ===")
print(f"ingredient_breakdown_df shape: {ingredient_breakdown_df.shape}")
print(f"Complete data available: {complete_data_mask.sum()}/{len(complete_data_mask)}")

complete_ingredients = ingredient_breakdown_df[complete_data_mask].copy()
print(f"Working with {len(complete_ingredients)} complete ingredient records")

ingredient_breakdown_df[complete_data_mask].head()


=== INGREDIENT BREAKDOWN DATA ===
ingredient_breakdown_df shape: (2572, 8)
Complete data available: 1895/2572
Working with 1895 complete ingredient records


,recipe_id,recipe_name,quantity,measurement,ingredient_name,original_text,original_quantity,original_measurement
0,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,60.0,g,cassava,"60 g cassava, boiled and blended",60.0,g
1,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,20.0,g,fish meat (milkfish),"20 g fish meat (milkfish), finely chopped",20.0,g
2,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,10.0,g,chicken meat,10 g chicken meat,10.0,g
3,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,5.0,g,coconut oil,5 g coconut oil,5.0,g
4,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,100.0,g,chicken broth,100 cc chicken broth,100,cc


In [ ]:
import pandas as pd

expected_cols = ['ingredient', 'energy_kcal_per_100g', 'carbs_g_per_100g',
                 'protein_g_per_100g', 'fat_g_per_100g', 'top_micronutrients']

# Check that data is a DataFrame and has required columns
if isinstance(combined_nutrition_data, pd.DataFrame) and all(col in combined_nutrition_data.columns for col in expected_cols):
    df = combined_nutrition_data[expected_cols]

    # --- Step 1: Drop rows where all 5 nutrient columns are empty/null/N/A ---
    nutrient_cols = ['energy_kcal_per_100g', 'carbs_g_per_100g', 'protein_g_per_100g', 'fat_g_per_100g', 'top_micronutrients']

    # Mark entries considered as "null"
    df[nutrient_cols] = df[nutrient_cols].replace(['', 'N/A', 'n/a', None], pd.NA)

    # Find rows where all nutrient fields are NA
    condition = df[nutrient_cols].isna().all(axis=1)

    # Save dropped rows and count
    dropped_rows = df[condition][['ingredient']].copy()
    dropped_rows['null_count'] = 5

    # Drop the identified rows
    df = df[~condition].reset_index(drop=True)

    # --- Step 2: Clean and convert numeric fields ---
    numeric_cols = ['energy_kcal_per_100g', 'carbs_g_per_100g', 'protein_g_per_100g', 'fat_g_per_100g']
    for col in numeric_cols:
        df[col] = df[col].replace(pd.NA, 0).fillna(0).astype(float)

else:
    print("❌ Could not load sheet or missing columns.")


C:\Users\Helena\AppData\Local\Temp\ipykernel_17276\4164426026.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nutrient_cols] = df[nutrient_cols].replace(['', 'N/A', 'n/a', None], pd.NA)


In [ ]:
print("✅ Cleaned DataFrame:")
display(df.head())
print("Current Shape:" , df.shape)

print("🗑️ Dropped rows where all nutrient info was missing:")
print(dropped_rows.count())

✅ Cleaned DataFrame:


,ingredient,energy_kcal_per_100g,carbs_g_per_100g,protein_g_per_100g,fat_g_per_100g,top_micronutrients
0,(0.8 fl oz) whole milk or infant formula,0.0,6.860,7.810,11.000,"Potassium, K, Calcium, Ca, Phosphorus, P"
1,(1 1/2 oz/45 g) frozen petite peas,0.0,5.180,3.380,0.950,"Potassium, K, Calcium, Ca, Phosphorus, P"
2,(1 heaped tbsp) green beans,40.0,7.410,1.970,0.275,"Potassium, K, Vitamin K (phylloquinone), Molyb..."
3,(1 kg) beef or lamb stew meat,237.0,-0.251,17.500,18.600,"Potassium, K, Phosphorus, P, Cholesterol"
4,(1.75 oz) celery stalk (about 1 stalk),16.7,3.320,0.492,0.162,"Potassium, K, Sodium, Na, Calcium, Ca"


Current Shape: (636, 6)
🗑️ Dropped rows where all nutrient info was missing:
ingredient    76
null_count    76
dtype: int64


In [ ]:
# Check measurements to ensure all are in grams
print("=== CHECKING MEASUREMENTS ===")
print("Unique measurement types in complete_ingredients:")
measurement_counts = complete_ingredients['measurement'].value_counts()
print(measurement_counts.head(10))

# Check if all measurements are in grams
non_gram_measurements = complete_ingredients[complete_ingredients['measurement'] != 'g']
print(f"\nNon-gram measurements: {len(non_gram_measurements)}")
if len(non_gram_measurements) > 0:
    print("Sample non-gram measurements:")
    print(non_gram_measurements[['ingredient_name', 'quantity', 'measurement']].head())
    
# For this analysis, we'll assume all measurements should be treated as grams
# If there are non-gram measurements, they should be converted beforehand
print(f"\nProceeding with assumption that all quantities are in grams or gram-equivalent units")

=== CHECKING MEASUREMENTS ===
Unique measurement types in complete_ingredients:
measurement
g            1548
heaped         16
knob           14
pinch          12
level          11
bowl           10
sprig          10
ripe            9
measuring       9
garlic          9
Name: count, dtype: int64

Non-gram measurements: 347
Sample non-gram measurements:
          ingredient_name quantity measurement
17             lemongrass        1       stalk
29                 pieces      4-6    broccoli
32  thick wholemeal bread        1       slice
61                  onion        1      spring
62                    ham        1       slice

Proceeding with assumption that all quantities are in grams or gram-equivalent units


In [ ]:
def find_nutrition_match(ingredient_name, nutrition_df):
    """
    Find nutrition match using 3-step strategy:
    1. Exact match (e.g., "sweet potato" matches exactly "sweet potato")
    2. Keyword match (e.g., "broccoli floret" matches "broccoli")
    3. No match - return None
    
    Includes custom rules:
    - Ignore case sensitivity
    - Remove text in brackets ()
    - Remove x000D characters
    - Special keyword matching rules
    - Exclusion rules for certain keywords
    """
    
    def clean_ingredient_name(name):
        """Clean ingredient name according to custom rules"""
        if pd.isna(name):
            return ""
        
        name = str(name)
        
        # Remove x000D
        name = name.replace('\x00\x0D', '').replace('x000D', '')
        
        # Remove text in brackets ()
        name = re.sub(r'\([^)]*\)', '', name)
        
        # Clean up extra spaces and convert to lowercase
        name = ' '.join(name.split()).lower().strip()
        
        return name
    
    def should_exclude_ingredient(ingredient_name):
        """Check if ingredient should be excluded from matching"""
        ingredient_lower = ingredient_name.lower()
        
        # Don't match hokkien or rice noodles
        if 'hokkien' in ingredient_lower or 'rice noodles' in ingredient_lower:
            return True
            
        # Don't match potato starch with potato (return all zeros)
        if 'potato starch' in ingredient_lower:
            return True
            
        return False
    
    def create_zero_nutrition():
        """Create a nutrition entry with all zeros for excluded ingredients"""
        # Create a pandas Series that matches the structure of nutrition_df
        zero_series = pd.Series({
            'ingredient': 'EXCLUDED - ALL ZEROS',
            'energy_kcal_per_100g': 0,
            'carbs_g_per_100g': 0,
            'protein_g_per_100g': 0,
            'fat_g_per_100g': 0,
            'top_micronutrients': ''
        })
        return zero_series, "excluded_zeros"
    
    def apply_special_matching_rules(ingredient_clean):
        """Apply special matching rules for specific ingredients"""
        # Yoghurt = yogurt
        if 'yoghurt' in ingredient_clean:
            ingredient_clean = ingredient_clean.replace('yoghurt', 'yogurt')
        
        # Cauliflower/broccoli = broccoli
        if 'cauliflower/broccoli' in ingredient_clean or 'cauliflower broccoli' in ingredient_clean:
            ingredient_clean = 'broccoli'
        
        # Oatmeal to baby oatmeal (if not already baby oatmeal)
        if 'oatmeal' in ingredient_clean and 'baby' not in ingredient_clean:
            ingredient_clean = ingredient_clean.replace('oatmeal', 'baby oatmeal')
        
        return ingredient_clean
    
    # Clean the input ingredient name
    ingredient_clean = clean_ingredient_name(ingredient_name)
    
    # Check exclusion rules first
    if should_exclude_ingredient(ingredient_clean):
        return create_zero_nutrition()
    
    # Special case for water - return all zeros
    if ingredient_clean in ['water', 'plain water', 'boiling water', 'cold water', 'warm water']:
        return create_zero_nutrition()
    
    # Apply special matching rules
    ingredient_clean = apply_special_matching_rules(ingredient_clean)
    
    # Clean nutrition dataframe ingredient names for comparison
    nutrition_df_clean = nutrition_df.copy()
    nutrition_df_clean['ingredient_clean'] = nutrition_df_clean['ingredient'].apply(clean_ingredient_name)
    
    # Step 1: Try exact match
    exact_match = nutrition_df_clean[nutrition_df_clean['ingredient_clean'] == ingredient_clean]
    if len(exact_match) > 0:
        return exact_match.iloc[0], "exact_match"
    
    # Step 2: Try keyword matching with special rules
    ingredient_words = ingredient_clean.split()
    
    for word in ingredient_words:
        if len(word) > 2:  # Only consider words longer than 2 characters
            
            # Special keyword matching rules
            if word == 'broth':
                # If there's "broth" in ingredient, match with "broth"
                try:
                    broth_match = nutrition_df_clean[nutrition_df_clean['ingredient_clean'].str.contains(r'\bbroth\b', na=False, regex=True)]
                    if len(broth_match) > 0:
                        return broth_match.iloc[0], f"keyword_match_broth"
                except:
                    pass
            
            elif 'long onion' in ingredient_clean:
                # Long onion should match with "long onion" specifically
                try:
                    long_onion_match = nutrition_df_clean[nutrition_df_clean['ingredient_clean'].str.contains('long onion', na=False, regex=False)]
                    if len(long_onion_match) > 0:
                        return long_onion_match.iloc[0], f"keyword_match_long_onion"
                except:
                    pass
            
            elif 'baby oatmeal' in ingredient_clean:
                # Baby oatmeal should match with "baby oatmeal" specifically
                try:
                    baby_oatmeal_match = nutrition_df_clean[nutrition_df_clean['ingredient_clean'].str.contains('baby oatmeal', na=False, regex=False)]
                    if len(baby_oatmeal_match) > 0:
                        return baby_oatmeal_match.iloc[0], f"keyword_match_baby_oatmeal"
                except:
                    pass
            
            # General keyword matching for other words
            try:
                # Escape special regex characters and use word boundaries for exact word matching
                pattern = r'\b' + re.escape(word) + r'\b'
                keyword_match = nutrition_df_clean[nutrition_df_clean['ingredient_clean'].str.contains(pattern, na=False, regex=True)]
                if len(keyword_match) > 0:
                    return keyword_match.iloc[0], f"keyword_match_{word}"
            except Exception as e:
                # If regex fails, try simple string contains
                try:
                    keyword_match = nutrition_df_clean[nutrition_df_clean['ingredient_clean'].str.contains(word, na=False, regex=False)]
                    if len(keyword_match) > 0:
                        return keyword_match.iloc[0], f"keyword_match_{word}"
                except:
                    continue
    
    # Step 3: No match found
    return None, "no_match"


In [ ]:
def calculate_ingredient_nutrition_new_strategy(ingredient_df, nutrition_df):
    """
    Calculate nutrition per ingredient using new strategy:
    - Use quantity directly (assume all in grams or gram-equivalent)
    - No conversion, direct proportional calculation from per 100g data
    """
    print("=== CALCULATING NUTRIENTS PER INGREDIENT (NEW STRATEGY) ===")
    
    result_df = ingredient_df.copy()
    
    # Add nutrition columns
    result_df['energy_kcal'] = 0.0
    result_df['carbs_g'] = 0.0
    result_df['protein_g'] = 0.0
    result_df['fat_g'] = 0.0
    result_df['micronutrients'] = ''
    result_df['nutrition_matched'] = False
    result_df['match_type'] = ''
    result_df['matched_ingredient'] = ''
    
    # Track statistics
    exact_matches = 0
    keyword_matches = 0
    no_matches = 0
    total_count = len(ingredient_df)
    
    print(f"Processing {total_count} ingredients...")
    
    for idx, row in ingredient_df.iterrows():
        ingredient_name = row['ingredient_name']
        quantity = row['quantity']
        
        # Find nutrition match
        nutrition_match, match_type = find_nutrition_match(ingredient_name, nutrition_df)
        
        if nutrition_match is not None:
            result_df.loc[idx, 'nutrition_matched'] = True
            result_df.loc[idx, 'match_type'] = match_type
            result_df.loc[idx, 'matched_ingredient'] = nutrition_match['ingredient']
            
            # Count match types
            if match_type == "exact_match":
                exact_matches += 1
            elif "keyword_match" in match_type:
                keyword_matches += 1
            
            # Calculate nutrition based on quantity (assume quantity is in grams)
            try:
                qty_numeric = float(quantity)
                if qty_numeric > 0:
                    # Calculate proportion of 100g
                    proportion = qty_numeric / 100.0
                    
                    # Scale numeric nutrients by proportion
                    if pd.notna(nutrition_match['energy_kcal_per_100g']):
                        result_df.loc[idx, 'energy_kcal'] = nutrition_match['energy_kcal_per_100g'] * proportion
                    if pd.notna(nutrition_match['carbs_g_per_100g']):
                        result_df.loc[idx, 'carbs_g'] = nutrition_match['carbs_g_per_100g'] * proportion
                    if pd.notna(nutrition_match['protein_g_per_100g']):
                        result_df.loc[idx, 'protein_g'] = nutrition_match['protein_g_per_100g'] * proportion
                    if pd.notna(nutrition_match['fat_g_per_100g']):
                        result_df.loc[idx, 'fat_g'] = nutrition_match['fat_g_per_100g'] * proportion
                    
                    # Store micronutrients as-is (don't scale)
                    if pd.notna(nutrition_match['top_micronutrients']):
                        result_df.loc[idx, 'micronutrients'] = str(nutrition_match['top_micronutrients'])
            except (ValueError, TypeError):
                # If quantity is not numeric, skip calculation but keep the match info
                pass
        else:
            no_matches += 1
            result_df.loc[idx, 'match_type'] = 'no_match'
    return result_df

In [ ]:
ingredients_with_nutrition_updated = calculate_ingredient_nutrition_new_strategy(complete_ingredients, df)

=== CALCULATING NUTRIENTS PER INGREDIENT (NEW STRATEGY) ===
Processing 1895 ingredients...


In [ ]:
# Check the total counts and math
total_processed = len(ingredients_with_nutrition_updated)
exact_matches = (ingredients_with_nutrition_updated['match_type'] == 'exact_match').sum()
keyword_matches = ingredients_with_nutrition_updated['match_type'].str.contains('keyword_match', na=False).sum()
no_matches = (ingredients_with_nutrition_updated['match_type'] == 'no_match').sum()
excluded_zeros = (ingredients_with_nutrition_updated['match_type'] == 'excluded_zeros').sum()

print(f"Total ingredients processed: {total_processed}")
print(f"Exact matches: {exact_matches}")
print(f"Keyword matches: {keyword_matches}")
print(f"No matches: {no_matches}")
print(f"Excluded zeros: {excluded_zeros}")

Total ingredients processed: 1895
Exact matches: 1496
Keyword matches: 251
No matches: 64
Excluded zeros: 84


In [ ]:
ingredients_with_nutrition_updated.columns.to_list
ingredients_with_nutrition_updated.head()

,recipe_id,recipe_name,quantity,measurement,ingredient_name,original_text,original_quantity,original_measurement,energy_kcal,carbs_g,protein_g,fat_g,micronutrients,nutrition_matched,match_type,matched_ingredient
0,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,60.0,g,cassava,"60 g cassava, boiled and blended",60.0,g,96.0,22.8600,0.816,0.168,"Potassium, K, Magnesium, Mg, Phosphorus, P",True,exact_match,cassava
1,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,20.0,g,fish meat (milkfish),"20 g fish meat (milkfish), finely chopped",20.0,g,0.0,0.0000,3.260,0.090,"Potassium, K, Phosphorus, P, Sodium, Na",True,exact_match,fish meat (milkfish)
2,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,10.0,g,chicken meat,10 g chicken meat,10.0,g,12.7,-0.0428,2.140,0.478,"Potassium, K, Phosphorus, P, Cholesterol",True,exact_match,chicken meat
3,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,5.0,g,coconut oil,5 g coconut oil,5.0,g,0.0,0.0420,0.000,4.955,"Total fat (NLEA), Fatty acids, total saturated...",True,exact_match,coconut oil
4,0,Cassava Porridge with Fish Sauce and Lemon (Bu...,100.0,g,chicken broth,100 cc chicken broth,100,cc,133.0,0.0000,17.900,7.160,"Potassium, K, Phosphorus, P, Cholesterol",True,keyword_match_chicken,(250 g) ground turkey or chicken


#### Validation Test

In [258]:
# NUTRITION CALCULATION VALIDATION FUNCTION
def validate_nutrition_calculations(calculated_df, nutrition_database_df, sample_size=20, tolerance=0.1):
    """
    Validate nutrition calculations by comparing calculated values with expected values
    from the main nutrition database.
    
    Parameters:
    - calculated_df: DataFrame with calculated nutrition values per ingredient
    - nutrition_database_df: Main nutrition database (df) with per 100g values
    - sample_size: Number of random samples to validate
    - tolerance: Acceptable percentage difference (0.1 = 10%)
    
    Returns:
    - validation_results: DataFrame with validation details
    - summary_stats: Dictionary with validation summary
    """
    
    def parse_quantity_safe(qty_str):
        """Safely parse quantity string including fractions"""
        if pd.isna(qty_str):
            return 0.0
        
        qty_str = str(qty_str)
        
        # Handle fractions
        fraction_map = {'½': 0.5, '¼': 0.25, '¾': 0.75, '⅓': 0.33, '⅔': 0.67, '⅛': 0.125}
        for frac, val in fraction_map.items():
            qty_str = qty_str.replace(frac, str(val))
        
        # Handle ranges (take average)
        if '-' in qty_str or '–' in qty_str:
            parts = re.split(r'[-–]', qty_str)
            if len(parts) == 2:
                try:
                    min_val = float(parts[0].strip())
                    max_val = float(parts[1].strip())
                    return (min_val + max_val) / 2
                except:
                    pass
        
        # Convert to float
        try:
            return float(qty_str)
        except:
            return 0.0
    
    print("🔍 NUTRITION CALCULATION VALIDATION")
    print("=" * 60)
    
    # Filter only ingredients that have nutrition matches (not excluded or no_match)
    matched_ingredients = calculated_df[
        (calculated_df['nutrition_matched'] == True) & 
        (calculated_df['match_type'] != 'excluded_zeros') &
        (calculated_df['match_type'] != 'no_match')
    ].copy()
    
    print(f"Total ingredients with nutrition matches: {len(matched_ingredients)}")
    
    # Sample random ingredients for validation
    if len(matched_ingredients) > sample_size:
        validation_sample = matched_ingredients.sample(n=sample_size, random_state=42)
        print(f"Validating random sample of {sample_size} ingredients")
    else:
        validation_sample = matched_ingredients.copy()
        print(f"Validating all {len(validation_sample)} matched ingredients")
    
    validation_results = []
    
    for idx, row in validation_sample.iterrows():
        ingredient_name = row['ingredient_name']
        matched_ingredient = row['matched_ingredient']
        quantity = parse_quantity_safe(row['quantity'])
        match_type = row['match_type']
        
        # Get calculated values
        calc_energy = row['energy_kcal'] if pd.notna(row['energy_kcal']) else 0
        calc_protein = row['protein_g'] if pd.notna(row['protein_g']) else 0
        calc_carbs = row['carbs_g'] if pd.notna(row['carbs_g']) else 0
        calc_fat = row['fat_g'] if pd.notna(row['fat_g']) else 0
        
        # Find the corresponding nutrition data in main database
        nutrition_match = nutrition_database_df[
            nutrition_database_df['ingredient'].str.lower().str.strip() == 
            matched_ingredient.lower().strip()
        ]
        
        if len(nutrition_match) > 0:
            nutr = nutrition_match.iloc[0]
            
            # Calculate expected values (quantity/100 * per_100g_value)
            if quantity > 0:
                proportion = quantity / 100.0
                
                expected_energy = nutr['energy_kcal_per_100g'] * proportion if pd.notna(nutr['energy_kcal_per_100g']) else 0
                expected_protein = nutr['protein_g_per_100g'] * proportion if pd.notna(nutr['protein_g_per_100g']) else 0
                expected_carbs = nutr['carbs_g_per_100g'] * proportion if pd.notna(nutr['carbs_g_per_100g']) else 0
                expected_fat = nutr['fat_g_per_100g'] * proportion if pd.notna(nutr['fat_g_per_100g']) else 0
                
                # Calculate percentage differences
                def calc_percentage_diff(calculated, expected):
                    if expected == 0 and calculated == 0:
                        return 0.0
                    elif expected == 0:
                        return float('inf') if calculated != 0 else 0.0
                    else:
                        return abs((calculated - expected) / expected) * 100
                
                energy_diff = calc_percentage_diff(calc_energy, expected_energy)
                protein_diff = calc_percentage_diff(calc_protein, expected_protein)
                carbs_diff = calc_percentage_diff(calc_carbs, expected_carbs)
                fat_diff = calc_percentage_diff(calc_fat, expected_fat)
                
                # Determine if validation passed (within tolerance)
                tolerance_percent = tolerance * 100
                energy_pass = energy_diff <= tolerance_percent or energy_diff == 0.0
                protein_pass = protein_diff <= tolerance_percent or protein_diff == 0.0
                carbs_pass = carbs_diff <= tolerance_percent or carbs_diff == 0.0
                fat_pass = fat_diff <= tolerance_percent or fat_diff == 0.0
                
                overall_pass = energy_pass and protein_pass and carbs_pass and fat_pass
                
                validation_results.append({
                    'ingredient_name': ingredient_name,
                    'matched_ingredient': matched_ingredient,
                    'match_type': match_type,
                    'quantity_g': quantity,
                    'proportion': proportion,
                    
                    # Energy validation
                    'calc_energy': round(calc_energy, 2),
                    'expected_energy': round(expected_energy, 2),
                    'energy_diff_percent': round(energy_diff, 2) if energy_diff != float('inf') else 'INF',
                    'energy_pass': energy_pass,
                    
                    # Protein validation
                    'calc_protein': round(calc_protein, 2),
                    'expected_protein': round(expected_protein, 2),
                    'protein_diff_percent': round(protein_diff, 2) if protein_diff != float('inf') else 'INF',
                    'protein_pass': protein_pass,
                    
                    # Carbs validation
                    'calc_carbs': round(calc_carbs, 2),
                    'expected_carbs': round(expected_carbs, 2),
                    'carbs_diff_percent': round(carbs_diff, 2) if carbs_diff != float('inf') else 'INF',
                    'carbs_pass': carbs_pass,
                    
                    # Fat validation
                    'calc_fat': round(calc_fat, 2),
                    'expected_fat': round(expected_fat, 2),
                    'fat_diff_percent': round(fat_diff, 2) if fat_diff != float('inf') else 'INF',
                    'fat_pass': fat_pass,
                    
                    'overall_pass': overall_pass,
                    'validation_status': 'PASS' if overall_pass else 'FAIL'
                })
            else:
                # Zero quantity case
                validation_results.append({
                    'ingredient_name': ingredient_name,
                    'matched_ingredient': matched_ingredient,
                    'match_type': match_type,
                    'quantity_g': quantity,
                    'validation_status': 'ZERO_QUANTITY',
                    'overall_pass': True  # Zero quantities are valid
                })
        else:
            # Could not find nutrition data for validation
            validation_results.append({
                'ingredient_name': ingredient_name,
                'matched_ingredient': matched_ingredient,
                'match_type': match_type,
                'quantity_g': quantity,
                'validation_status': 'NO_DB_MATCH',
                'overall_pass': False
            })
    
    # Create validation results DataFrame
    validation_df = pd.DataFrame(validation_results)
    
    # Calculate summary statistics
    if len(validation_df) > 0:
        total_validations = len(validation_df)
        successful_validations = len(validation_df[validation_df['validation_status'].isin(['PASS', 'FAIL'])])
        passed_validations = (validation_df['overall_pass'] == True).sum()
        failed_validations = (validation_df['validation_status'] == 'FAIL').sum()
        no_db_match = (validation_df['validation_status'] == 'NO_DB_MATCH').sum()
        zero_quantity = (validation_df['validation_status'] == 'ZERO_QUANTITY').sum()
        
        # Calculate pass rates for each nutrient (only for PASS/FAIL status)
        valid_for_nutrient_check = validation_df[validation_df['validation_status'].isin(['PASS', 'FAIL'])]
        if len(valid_for_nutrient_check) > 0:
            energy_pass_rate = (valid_for_nutrient_check['energy_pass'] == True).sum() / len(valid_for_nutrient_check) * 100
            protein_pass_rate = (valid_for_nutrient_check['protein_pass'] == True).sum() / len(valid_for_nutrient_check) * 100
            carbs_pass_rate = (valid_for_nutrient_check['carbs_pass'] == True).sum() / len(valid_for_nutrient_check) * 100
            fat_pass_rate = (valid_for_nutrient_check['fat_pass'] == True).sum() / len(valid_for_nutrient_check) * 100
        else:
            energy_pass_rate = protein_pass_rate = carbs_pass_rate = fat_pass_rate = 0
        
        summary_stats = {
            'total_validations': total_validations,
            'successful_validations': successful_validations,
            'passed_validations': passed_validations,
            'failed_validations': failed_validations,
            'no_db_match': no_db_match,
            'zero_quantity': zero_quantity,
            'overall_pass_rate': passed_validations / total_validations * 100 if total_validations > 0 else 0,
            'energy_pass_rate': energy_pass_rate,
            'protein_pass_rate': protein_pass_rate,
            'carbs_pass_rate': carbs_pass_rate,
            'fat_pass_rate': fat_pass_rate,
            'tolerance_percent': tolerance * 100
        }
    else:
        summary_stats = {'error': 'No validations performed'}
    
    return validation_df, summary_stats

# Run the validation
print("Starting nutrition calculation validation...")
validation_results, validation_summary = validate_nutrition_calculations(
    ingredients_with_nutrition_updated, 
    df, 
    sample_size=30,  # Validate 30 random samples
    tolerance=0.05   # 5% tolerance
)

Starting nutrition calculation validation...
🔍 NUTRITION CALCULATION VALIDATION
Total ingredients with nutrition matches: 1747
Validating random sample of 30 ingredients


In [259]:
# Display validation results and summary
print("📊 VALIDATION SUMMARY REPORT")
print("=" * 60)

if 'error' in validation_summary:
    print(f"❌ Validation Error: {validation_summary['error']}")
else:
    # Display summary statistics
    print(f"📈 OVERALL VALIDATION STATISTICS:")
    print(f"   Total validations performed: {validation_summary['total_validations']}")
    print(f"   Successful validations: {validation_summary['successful_validations']}")
    print(f"   Passed validations: {validation_summary['passed_validations']}")
    print(f"   Failed validations: {validation_summary['failed_validations']}")
    print(f"   No database match: {validation_summary['no_db_match']}")
    print(f"   Zero quantity cases: {validation_summary['zero_quantity']}")
    print(f"   Overall pass rate: {validation_summary['overall_pass_rate']:.1f}%")
    print(f"   Tolerance used: ±{validation_summary['tolerance_percent']}%")
    
    print(f"\n🧪 NUTRIENT-SPECIFIC PASS RATES:")
    print(f"   Energy (kcal): {validation_summary['energy_pass_rate']:.1f}%")
    print(f"   Protein (g): {validation_summary['protein_pass_rate']:.1f}%")
    print(f"   Carbohydrates (g): {validation_summary['carbs_pass_rate']:.1f}%")
    print(f"   Fat (g): {validation_summary['fat_pass_rate']:.1f}%")

# Display detailed validation results
print(f"\n📋 DETAILED VALIDATION RESULTS:")
print("-" * 60)

if len(validation_results) > 0:
    # Show passed validations
    passed_validations = validation_results[validation_results['validation_status'] == 'PASS']
    if len(passed_validations) > 0:
        print(f"\n✅ PASSED VALIDATIONS ({len(passed_validations)}):")
        for idx, row in passed_validations.head(5).iterrows():
            print(f"   • {row['ingredient_name']} ({row['quantity_g']}g)")
            print(f"     Matched: {row['matched_ingredient']} via {row['match_type']}")
            print(f"     Energy: {row['calc_energy']} vs {row['expected_energy']} (diff: {row['energy_diff_percent']}%)")
    
    # Show failed validations
    failed_validations = validation_results[validation_results['validation_status'] == 'FAIL']
    if len(failed_validations) > 0:
        print(f"\n❌ FAILED VALIDATIONS ({len(failed_validations)}):")
        for idx, row in failed_validations.head(5).iterrows():
            print(f"   • {row['ingredient_name']} ({row['quantity_g']}g)")
            print(f"     Matched: {row['matched_ingredient']} via {row['match_type']}")
            print(f"     Energy: {row['calc_energy']} vs {row['expected_energy']} (diff: {row['energy_diff_percent']}%)")
            failed_nutrients = []
            if not row['energy_pass']: failed_nutrients.append('Energy')
            if not row['protein_pass']: failed_nutrients.append('Protein')
            if not row['carbs_pass']: failed_nutrients.append('Carbs')
            if not row['fat_pass']: failed_nutrients.append('Fat')
            print(f"     Failed nutrients: {', '.join(failed_nutrients)}")
    
    # Show some examples in tabular format
    print(f"\n📊 SAMPLE VALIDATION DETAILS:")
    display_cols = ['ingredient_name', 'quantity_g', 'calc_energy', 'expected_energy', 
                   'energy_diff_percent', 'validation_status']
    sample_results = validation_results[display_cols].head(10)
    print(sample_results.to_string(index=False))

# Overall validation assessment
print(f"\n🎯 VALIDATION ASSESSMENT:")
if 'error' not in validation_summary:
    overall_rate = validation_summary['overall_pass_rate']
    if overall_rate >= 95:
        print("   🟢 EXCELLENT: Calculations are highly accurate (≥95% pass rate)")
    elif overall_rate >= 90:
        print("   🟡 GOOD: Calculations are mostly accurate (≥90% pass rate)")
    elif overall_rate >= 80:
        print("   🟠 FAIR: Some calculation issues detected (≥80% pass rate)")
    else:
        print("   🔴 POOR: Significant calculation errors detected (<80% pass rate)")
        
    print(f"   📐 Calculation accuracy: {overall_rate:.1f}%")
    print(f"   🔍 Tolerance: ±{validation_summary['tolerance_percent']}%")

print(f"\n" + "=" * 60)
print("✅ VALIDATION COMPLETE!")

📊 VALIDATION SUMMARY REPORT
📈 OVERALL VALIDATION STATISTICS:
   Total validations performed: 30
   Successful validations: 30
   Passed validations: 30
   Failed validations: 0
   No database match: 0
   Zero quantity cases: 0
   Overall pass rate: 100.0%
   Tolerance used: ±5.0%

🧪 NUTRIENT-SPECIFIC PASS RATES:
   Energy (kcal): 100.0%
   Protein (g): 100.0%
   Carbohydrates (g): 100.0%
   Fat (g): 100.0%

📋 DETAILED VALIDATION RESULTS:
------------------------------------------------------------

✅ PASSED VALIDATIONS (30):
   • tuna in spring water (185.0g)
     Matched: tuna in spring water via exact_match
     Energy: 0.0 vs 0.0 (diff: 0.0%)
   • butter  _x000D_ (20.0g)
     Matched: (10g) Butter via keyword_match_butter
     Energy: 129.0 vs 129.0 (diff: 0.0%)
   • sweet potato (50.0g)
     Matched: sweet potato via exact_match
     Energy: 39.5 vs 39.5 (diff: 0.0%)
   • unsalted butter (15.0g)
     Matched: (15g) unsalted butter via exact_match
     Energy: 0.0 vs 0.0 (diff: 0.0%

In [ ]:
import pandas as pd

# Show all exact matches
exact_matches_df = ingredients_with_nutrition_udpated[ingredients_with_nutrition_new['match_type'] == 'exact_match']
print(f"\nExact Matches:")

exact_sample = exact_matches_df[['ingredient_name', 'matched_ingredient', 'match_type']]

# Display all rows without being cut off
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(exact_sample)


In [ ]:
keyword_matches_df = ingredients_with_nutrition_updated [ingredients_with_nutrition_updated ['match_type'].str.contains('keyword_match', na=False)]
keyword_matches = keyword_matches_df[['ingredient_name', 'matched_ingredient', 'match_type']]

# Ensure full display
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(keyword_matches)

In [ ]:
# Show all no matches
no_matches_df = ingredients_with_nutrition_updated [ingredients_with_nutrition_updated['match_type'] == 'no_match']
print(f"\nNo Matches:")

no_match_sample = no_matches_df[['ingredient_name', 'match_type']]

# Display all rows and columns fully
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(no_match_sample)


## Calculate Recipe

In [261]:
# 🔧 SOLUTION: Ensure no_match cases have zero values for all numeric fields
def ensure_no_match_zero_values(df):
    print("🔧 ENSURING NO_MATCH CASES HAVE ZERO VALUES")
    print("=" * 50)
    
    # Define all numeric nutrition columns that should be zero for no_match
    numeric_nutrition_cols = [
        'energy_kcal', 'carbs_g', 'protein_g', 'fat_g', 
        'carbohydrate_g',  # Alternative naming
        # Add any other numeric nutrition columns you have
    ]
    
    # Find no_match cases
    no_match_mask = df['match_type'] == 'no_match'
    no_match_count = no_match_mask.sum()
    
    print(f"📊 Found {no_match_count} ingredients with 'no_match' status")
    
    if no_match_count > 0:
        # Set all numeric nutrition values to 0 for no_match cases
        for col in numeric_nutrition_cols:
            if col in df.columns:
                # Check current state
                before_none = df.loc[no_match_mask, col].isna().sum()
                before_values = df.loc[no_match_mask, col].notna().sum()
                
                # Set to zero
                df.loc[no_match_mask, col] = 0.0
                
                print(f"   ✅ {col}: Set {before_none} None/NaN + {before_values} existing values → 0")
        
        # Also ensure nutrition_matched is False for no_match cases
        if 'nutrition_matched' in df.columns:
            df.loc[no_match_mask, 'nutrition_matched'] = False
            print(f"   ✅ nutrition_matched: Set to False for all no_match cases")
        
        # Set micronutrients to empty list for no_match cases
        if 'micronutrients' in df.columns:
            df.loc[no_match_mask, 'micronutrients'] = '[]'  # Empty list as string
            print(f"   ✅ micronutrients: Set to empty list for all no_match cases")
        
        # Show sample of corrected data
        print(f"\n📋 SAMPLE CORRECTED NO_MATCH CASES:")
        sample_cols = ['ingredient_name', 'match_type'] + [col for col in numeric_nutrition_cols if col in df.columns]
        sample_no_match = df[no_match_mask][sample_cols].head(3)
        display(sample_no_match)
        
        print(f"\n✅ SUCCESS: All {no_match_count} no_match cases now have zero values for numeric fields!")
    else:
        print("ℹ️ No ingredients with 'no_match' status found.")
    
    return df

# Apply the correction to your current data
if 'ingredients_with_nutrition_updated' in locals():
    print("🧪 TESTING: Before correction")
    no_match_before = ingredients_with_nutrition_updated[ingredients_with_nutrition_updated['match_type'] == 'no_match']
    if len(no_match_before) > 0:
        print(f"Sample before correction (showing nutrition values):")
        display(no_match_before[['ingredient_name', 'match_type', 'energy_kcal', 'carbs_g', 'protein_g', 'fat_g']].head(3))
    
    # Apply the correction
    ingredients_with_nutrition_updated = ensure_no_match_zero_values(ingredients_with_nutrition_updated)
    
    print(f"\n🧪 VERIFICATION: After correction")
    no_match_after = ingredients_with_nutrition_updated[ingredients_with_nutrition_updated['match_type'] == 'no_match']
    if len(no_match_after) > 0:
        print(f"Sample after correction (all should be 0.0):")
        display(no_match_after[['ingredient_name', 'match_type', 'energy_kcal', 'carbs_g', 'protein_g', 'fat_g']].head(3))
else:
    print("❌ ingredients_with_nutrition_updated not found. Please run the ingredient processing first.")

🧪 TESTING: Before correction
Sample before correction (showing nutrition values):


,ingredient_name,match_type,energy_kcal,carbs_g,protein_g,fat_g
17,lemongrass,no_match,0.0,0.0,0.0,0.0
34,polenta (a coarse,no_match,0.0,0.0,0.0,0.0
117,breadcrumbs,no_match,0.0,0.0,0.0,0.0


🔧 ENSURING NO_MATCH CASES HAVE ZERO VALUES
📊 Found 64 ingredients with 'no_match' status
   ✅ energy_kcal: Set 0 None/NaN + 64 existing values → 0
   ✅ carbs_g: Set 0 None/NaN + 64 existing values → 0
   ✅ protein_g: Set 0 None/NaN + 64 existing values → 0
   ✅ fat_g: Set 0 None/NaN + 64 existing values → 0
   ✅ nutrition_matched: Set to False for all no_match cases
   ✅ micronutrients: Set to empty list for all no_match cases

📋 SAMPLE CORRECTED NO_MATCH CASES:


,ingredient_name,match_type,energy_kcal,carbs_g,protein_g,fat_g
17,lemongrass,no_match,0.0,0.0,0.0,0.0
34,polenta (a coarse,no_match,0.0,0.0,0.0,0.0
117,breadcrumbs,no_match,0.0,0.0,0.0,0.0



✅ SUCCESS: All 64 no_match cases now have zero values for numeric fields!

🧪 VERIFICATION: After correction
Sample after correction (all should be 0.0):


,ingredient_name,match_type,energy_kcal,carbs_g,protein_g,fat_g
17,lemongrass,no_match,0.0,0.0,0.0,0.0
34,polenta (a coarse,no_match,0.0,0.0,0.0,0.0
117,breadcrumbs,no_match,0.0,0.0,0.0,0.0


In [264]:
# Calculate recipe-level nutrition using new strategy
def calculate_recipe_nutrition_new(ingredients_df):
    """
    Calculate overall recipe nutrition by aggregating ingredient nutrition.
    For numeric nutrients: sum all values and divide by number of ingredients
    For micronutrients: find overlapping nutrients across all ingredients
    """
    print("=== CALCULATING RECIPE-LEVEL NUTRITION (NEW STRATEGY) ===")
    
    # Group by recipe
    recipe_nutrition = []
    
    for recipe_name, recipe_group in ingredients_df.groupby('recipe_name'):
        # Filter only ingredients with nutrition data
        with_nutrition = recipe_group[recipe_group['nutrition_matched'] == True]
        
        if len(with_nutrition) == 0:
            continue
            
        recipe_data = {
            'recipe_name': recipe_name,
            'total_ingredients': len(recipe_group),
            'ingredients_with_nutrition': len(with_nutrition),
            'nutrition_coverage': len(with_nutrition) / len(recipe_group) * 100,
            'exact_matches': len(with_nutrition[with_nutrition['match_type'] == 'exact_match']),
            'keyword_matches': len(with_nutrition[with_nutrition['match_type'].str.contains('keyword_match', na=False)])
        }
        
        # Calculate total numeric nutrients (sum without averaging)
        numeric_cols = ['energy_kcal', 'carbs_g', 'protein_g', 'fat_g']
        
        for col in numeric_cols:
            total_value = with_nutrition[col].sum()
            recipe_data[f'total_{col}'] = total_value  # Total nutrition value
            # Optional: Keep average per ingredient with nutrition data
            recipe_data[f'avg_per_matched_{col}'] = total_value / len(with_nutrition) if len(with_nutrition) > 0 else 0
        
        # Find overlapping micronutrients
        micronutrient_lists = []
        for _, ingredient in with_nutrition.iterrows():
            if ingredient['micronutrients'] and str(ingredient['micronutrients']).strip():
                try:
                    # Try to parse as list if it's a string representation
                    micro_str = str(ingredient['micronutrients'])
                    if micro_str.startswith('[') and micro_str.endswith(']'):
                        micro_list = eval(micro_str)  # Be careful with eval in production
                    else:
                        # Split by comma if it's a comma-separated string
                        micro_list = [m.strip() for m in micro_str.split(',')]
                    micronutrient_lists.append(set(micro_list))
                except:
                    # If parsing fails, treat as single item
                    micronutrient_lists.append({str(ingredient['micronutrients'])})
        
        # Find overlapping micronutrients (present in all ingredients)
        if micronutrient_lists:
            overlapping_micronutrients = set.intersection(*micronutrient_lists)
            all_micronutrients = set.union(*micronutrient_lists)
            
            recipe_data['overlapping_micronutrients'] = list(overlapping_micronutrients)
            recipe_data['all_micronutrients'] = list(all_micronutrients)
            recipe_data['micronutrient_overlap_count'] = len(overlapping_micronutrients)
            recipe_data['total_unique_micronutrients'] = len(all_micronutrients)
        else:
            recipe_data['overlapping_micronutrients'] = []
            recipe_data['all_micronutrients'] = []
            recipe_data['micronutrient_overlap_count'] = 0
            recipe_data['total_unique_micronutrients'] = 0
        
        recipe_nutrition.append(recipe_data)
    
    recipe_nutrition_df = pd.DataFrame(recipe_nutrition)
    print(f"✅ Calculated nutrition for {len(recipe_nutrition_df)} recipes")
    
    return recipe_nutrition_df

In [266]:
# Execute recipe nutrition calculation
recipe_nutrition_new = calculate_recipe_nutrition_new(ingredients_with_nutrition_updated)

print("columns:" , recipe_nutrition_new.columns.to_list)

display(recipe_nutrition_new.head(10))


=== CALCULATING RECIPE-LEVEL NUTRITION (NEW STRATEGY) ===
✅ Calculated nutrition for 403 recipes
columns: <bound method IndexOpsMixin.tolist of Index(['recipe_name', 'total_ingredients', 'ingredients_with_nutrition',
       'nutrition_coverage', 'exact_matches', 'keyword_matches',
       'total_energy_kcal', 'avg_per_matched_energy_kcal', 'total_carbs_g',
       'avg_per_matched_carbs_g', 'total_protein_g',
       'avg_per_matched_protein_g', 'total_fat_g', 'avg_per_matched_fat_g',
       'overlapping_micronutrients', 'all_micronutrients',
       'micronutrient_overlap_count', 'total_unique_micronutrients'],
      dtype='object')>
✅ Calculated nutrition for 403 recipes
columns: <bound method IndexOpsMixin.tolist of Index(['recipe_name', 'total_ingredients', 'ingredients_with_nutrition',
       'nutrition_coverage', 'exact_matches', 'keyword_matches',
       'total_energy_kcal', 'avg_per_matched_energy_kcal', 'total_carbs_g',
       'avg_per_matched_carbs_g', 'total_protein_g',
       '

,recipe_name,total_ingredients,ingredients_with_nutrition,nutrition_coverage,exact_matches,keyword_matches,total_energy_kcal,avg_per_matched_energy_kcal,total_carbs_g,avg_per_matched_carbs_g,total_protein_g,avg_per_matched_protein_g,total_fat_g,avg_per_matched_fat_g,overlapping_micronutrients,all_micronutrients,micronutrient_overlap_count,total_unique_micronutrients
0,100% fruity smoothie,4,4,100.0,4,0,65.02126,16.255315,28.234355,7.058589,16.632625,4.158156,22.31776,5.579440,"[K, Potassium]","[Calcium, Citric acid, Potassium, Malic acid, ...",2,8
1,African bean stew recipe,8,8,100.0,6,1,120.48000,15.060000,30.535500,3.816937,24.088500,3.011062,12.91225,1.614031,[],"[Vitamin K, Calcium, B, Molybdenum, Sodium, Po...",0,16
2,African stew recipe,7,7,100.0,5,1,178.80000,25.542857,25.374300,3.624900,36.820500,5.260071,16.04125,2.291607,[],"[Vitamin K, Calcium, B, Cholesterol, Molybdenu...",0,17
3,African sweet potato stew recipe,11,11,100.0,9,1,344.75000,31.340909,84.963000,7.723909,65.519000,5.956273,26.60850,2.418955,[],"[Vitamin K, Calcium, Vitamin C, B, Ca, Molybde...",0,20
4,"Alphabet Pasta with Fish, Tomato, and Potato",6,6,100.0,5,0,203.70000,33.950000,21.421800,3.570300,37.478200,6.246367,12.20210,2.033683,[],"[Iron, Vitamin K, Calcium, Ca, Sodium, Fe, K, ...",0,12
5,Anchovy Chayote Porridge (Bubur Teri Nasi Siam),5,5,100.0,5,0,215.22300,43.044600,49.230390,9.846078,16.792310,3.358462,19.60830,3.921660,[],"[Folate, food, Total fat (NLEA), Starch, Fatty...",0,14
6,Apple Banana Yogurt Puree,1,1,100.0,0,1,115.50000,115.500000,24.150000,24.150000,1.587000,1.587000,1.39200,1.392000,"[Mg, Magnesium, K, Potassium, P, Phosphorus]","[Mg, Magnesium, K, Potassium, P, Phosphorus]",6,6
7,Apple Crumble,6,6,100.0,5,1,1009.60000,168.266667,290.909000,48.484833,24.750500,4.125083,45.43200,7.572000,[],"[Calcium, Retinol, Magnesium, Molybdenum, by s...",0,17
8,Apple and blueberry stew,3,3,100.0,2,0,86.74000,28.913333,20.160000,6.720000,0.507700,0.169233,0.46960,0.156533,"[K, Potassium]","[Calcium, Vitamin C, total ascorbic acid, Mali...",2,9
9,Apple and orange compote,1,1,100.0,1,0,14.16000,14.160000,3.090000,3.090000,0.220200,0.220200,0.09750,0.097500,"[K, Citric acid, Potassium, Malic acid]","[K, Citric acid, Potassium, Malic acid]",4,4


In [267]:
# 🧪 DETAILED VALIDATION TESTING FOR 5 RECIPES
print("=" * 80)
print("🧪 DETAILED VALIDATION TESTING FOR 5 SPECIFIC RECIPES")
print("=" * 80)

# Select 5 recipes for detailed validation
test_recipes = recipe_nutrition_new.head(5)['recipe_name'].tolist()

print(f"Testing these recipes: {test_recipes}")
print("\n" + "="*50)

for i, recipe_name in enumerate(test_recipes, 1):
    print(f"\n🔍 RECIPE {i}: {recipe_name}")
    print("-" * 60)
    
    # Get recipe ingredients
    recipe_ingredients = ingredients_with_nutrition_updated[
        ingredients_with_nutrition_updated['recipe_name'] == recipe_name
    ]
    
    # Get recipe nutrition results
    recipe_nutrition = recipe_nutrition_new[
        recipe_nutrition_new['recipe_name'] == recipe_name
    ].iloc[0]
    
    print(f"📊 RECIPE OVERVIEW:")
    print(f"   • Total ingredients: {len(recipe_ingredients)}")
    print(f"   • Matched ingredients: {recipe_nutrition['ingredients_with_nutrition']}")
    print(f"   • Nutrition coverage: {recipe_nutrition['nutrition_coverage']:.1f}%")
    print(f"   • Exact matches: {recipe_nutrition['exact_matches']}")
    print(f"   • Keyword matches: {recipe_nutrition['keyword_matches']}")
    
    # Show ingredient breakdown
    print(f"\n🥄 INGREDIENT BREAKDOWN:")
    ingredient_details = []
    total_manual_energy = 0
    total_manual_protein = 0
    total_manual_carbs = 0
    total_manual_fat = 0
    
    for _, ingredient in recipe_ingredients.iterrows():
        has_nutrition = ingredient['nutrition_matched']
        energy = ingredient['energy_kcal'] if has_nutrition else 0
        protein = ingredient['protein_g'] if has_nutrition else 0
        carbs = ingredient['carbs_g'] if has_nutrition else 0
        fat = ingredient['fat_g'] if has_nutrition else 0
        
        if has_nutrition:
            total_manual_energy += energy
            total_manual_protein += protein
            total_manual_carbs += carbs
            total_manual_fat += fat
        
        status = "✅ Matched" if has_nutrition else "❌ No match"
        print(f"   • {ingredient['ingredient_name']:<25} | {status:<12} | "
              f"Energy: {energy:>6.1f} kcal | Protein: {protein:>5.1f}g | "
              f"Carbs: {carbs:>5.1f}g | Fat: {fat:>5.1f}g")
    
    # Verify calculations
    print(f"\n🧮 MANUAL vs SYSTEM CALCULATION VERIFICATION:")
    print(f"   Energy:  Manual={total_manual_energy:>6.1f} | System={recipe_nutrition['total_energy_kcal']:>6.1f} | Match: {'✅' if abs(total_manual_energy - recipe_nutrition['total_energy_kcal']) < 0.1 else '❌'}")
    print(f"   Protein: Manual={total_manual_protein:>6.1f} | System={recipe_nutrition['total_protein_g']:>6.1f} | Match: {'✅' if abs(total_manual_protein - recipe_nutrition['total_protein_g']) < 0.1 else '❌'}")
    print(f"   Carbs:   Manual={total_manual_carbs:>6.1f} | System={recipe_nutrition['total_carbs_g']:>6.1f} | Match: {'✅' if abs(total_manual_carbs - recipe_nutrition['total_carbs_g']) < 0.1 else '❌'}")
    print(f"   Fat:     Manual={total_manual_fat:>6.1f} | System={recipe_nutrition['total_fat_g']:>6.1f} | Match: {'✅' if abs(total_manual_fat - recipe_nutrition['total_fat_g']) < 0.1 else '❌'}")
    
    # Show micronutrients
    if recipe_nutrition['total_unique_micronutrients'] > 0:
        print(f"\n🧬 MICRONUTRIENTS:")
        print(f"   • Total unique: {recipe_nutrition['total_unique_micronutrients']}")
        print(f"   • Overlapping: {recipe_nutrition['micronutrient_overlap_count']}")
        if recipe_nutrition['overlapping_micronutrients']:
            overlap_str = str(recipe_nutrition['overlapping_micronutrients'])[:100]
            print(f"   • Common nutrients: {overlap_str}{'...' if len(str(recipe_nutrition['overlapping_micronutrients'])) > 100 else ''}")
    
    print("-" * 60)

print(f"\n✅ VALIDATION COMPLETE FOR ALL {len(test_recipes)} RECIPES")
print("=" * 80)

🧪 DETAILED VALIDATION TESTING FOR 5 SPECIFIC RECIPES
Testing these recipes: ['100% fruity smoothie', 'African bean stew recipe', 'African stew recipe', 'African sweet potato stew recipe', 'Alphabet Pasta with Fish, Tomato, and Potato']


🔍 RECIPE 1: 100% fruity smoothie
------------------------------------------------------------
📊 RECIPE OVERVIEW:
   • Total ingredients: 4
   • Matched ingredients: 4
   • Nutrition coverage: 100.0%
   • Exact matches: 4
   • Keyword matches: 0

🥄 INGREDIENT BREAKDOWN:
   • blueberries               | ✅ Matched    | Energy:   12.8 kcal | Protein:   0.1g | Carbs:   2.9g | Fat:   0.1g
   • raspberry                 | ✅ Matched    | Energy:   28.6 kcal | Protein:   0.5g | Carbs:   6.5g | Fat:   0.1g
   • fresh orange juice        | ✅ Matched    | Energy:   23.6 kcal | Protein:   0.4g | Carbs:   5.2g | Fat:   0.2g
   • growing-up milk formula   | ✅ Matched    | Energy:    0.0 kcal | Protein:  15.6g | Carbs:  13.7g | Fat:  22.0g

🧮 MANUAL vs SYSTEM CALCULAT